# Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
import plotly.colors as pc
from plotly.subplots import make_subplots

import datetime as dt
from pathlib import Path
import glob
import os
import time
import psutil
import random

from scipy.io import loadmat
from scipy.signal import savgol_filter

from tqdm import tqdm
import chardet
import joblib
import json

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score, KFold, TimeSeriesSplit
from sklearn.multioutput import MultiOutputRegressor

import optuna
from optuna.pruners import MedianPruner
from optuna.trial import FixedTrial

# Jupyter magic command 
%matplotlib inline


In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa

# Optionally enable memory growth (recommended if you experience GPU memory allocation issues)
#gpus = tf.config.list_physical_devices('GPU')
#if gpus:
 #   try:
        # Enable memory growth for each GPU
 #       for gpu in gpus:
 #           tf.config.experimental.set_memory_growth(gpu, True)
 #   except RuntimeError as e:
 #       print(e)

# Import Keras components including the LSTM layer 
# (in TF 2.x, LSTM is GPU-optimized if possible)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input, Reshape , Dropout , Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint , Callback
from tensorflow.keras.models import load_model
from tensorflow.keras import mixed_precision
from tensorflow.keras.models import clone_model
from tensorflow.keras.optimizers.schedules import ExponentialDecay


# Optionally, for explicit GPU-optimized LSTM (useful if you're using an older TF version):21
# from tensorflow.keras.layers import CuDNNLSTM

# For monitoring GPU availability (optional)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:

# Set environment variable to disable GPU (force using CPU)
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# Now, TensorFlow will use the CPU for computations, regardless of GPU availability.
# Optionally, for monitoring GPU availability (optional)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


In [ ]:
# 1. Set random seed for built-in random module
random.seed(42)

# 2. Set random seed for NumPy
np.random.seed(42)

# 3. Set random seed for TensorFlow
tf.random.set_seed(42)


# Functions

In [ ]:
import numpy as np
import pandas as pd

def create_exog_sequences(
    input_df: pd.DataFrame,
    output_df: pd.DataFrame,
    input_cols: list,
    output_cols: list,
    nb: int,
    nf: int
):
    """
    Build sliding windows of exogenous inputs + floater motion targets.
    Returns only X and Y—no per-window init_vals, since we'll do a warm start.

    Args:
      input_df: DataFrame with your input features (e.g. wave elev), shape (N, ...)
      output_df: DataFrame with your target features, shape (N, num_outputs)
      input_cols: list of column names in input_df to use
      output_cols: list of column names in output_df to predict
      nb: number of past lags
      nf: number of future steps

    Returns:
      X: np.ndarray, shape (samples, nb+nf, len(input_cols))
      Y: np.ndarray, shape (samples, nf, len(output_cols))
    """
    L= nb + 1 + nf                       # total window length    
    Xs, Ys = [], []
    X_arr = input_df[input_cols].to_numpy()
    Y_arr = output_df[output_cols].to_numpy()
    N = len(X_arr)

    for i in range(nb, N - nf):
        window = X_arr[i - nb : i + nf + 1]   # shape (L, features)
        Xs.append(window)
        Ys.append(Y_arr[i])                  # single row of shape (outputs,)

    X = np.stack(Xs, axis=0)   # (num_samples, L, features)
    Y = np.stack(Ys, axis=0)   # (num_samples, outputs)
    return X, Y


In [ ]:
def create_sequences(X, y, time_steps, future_steps):
    if len(X) < time_steps + future_steps:
        raise ValueError("Not enough data for the given time steps and future steps!")

    Xs, ys = [], []
    for i in range(len(X) - time_steps - future_steps + 1):  # Adjusted index range
        Xs.append(X.iloc[i:i + time_steps].values)  # Use `.iloc` for Pandas indexing
        ys.append(y.iloc[i + time_steps:i + time_steps + future_steps].values)  # Predict `future_steps`
        
    return np.array(Xs), np.array(ys)

# Loading Data

In [ ]:
# load data
df_train_full = pd.read_csv('prepared_data/train_data.csv')
df_val_full = pd.read_csv('prepared_data/val_data.csv')
df_test_full = pd.read_csv('prepared_data/test_data.csv')

print(df_train_full.head())
print(df_val_full.head())
print(df_test_full.head())


In [ ]:
# define test case
case='Tp6p8s_Hs2m'
df_case_train = df_train_full[df_train_full['test_name'] == case].copy()
df_case_val = df_val_full[df_val_full['test_name'] == case].copy()
df_case_test = df_test_full[df_test_full['test_name'] == case].copy()


# Scaling Data

In [ ]:
# Initialize scalers
scaler_X = MinMaxScaler(feature_range=(-1, 1))
scaler_y = MinMaxScaler(feature_range=(-1, 1))

# Feature and target columns
input_cols = ['eta']
output_cols = ['heave', 'pitch', 'pendulum']

# Extract training data as DataFrames
X_train = df_train_full[input_cols]
y_train = df_train_full[output_cols]

# Fit scalers
scaler_X.fit(X_train)
scaler_y.fit(y_train)

# Transform all sets (keeps DataFrame structure)
X_train_scaled = pd.DataFrame(scaler_X.transform(X_train), columns=input_cols)
y_train_scaled = pd.DataFrame(scaler_y.transform(y_train), columns=output_cols)

X_val_scaled = pd.DataFrame(scaler_X.transform(df_val_full[input_cols]), columns=input_cols)
y_val_scaled = pd.DataFrame(scaler_y.transform(df_val_full[output_cols]), columns=output_cols)

X_test_scaled = pd.DataFrame(scaler_X.transform(df_test_full[input_cols]), columns=input_cols)
y_test_scaled = pd.DataFrame(scaler_y.transform(df_test_full[output_cols]), columns=output_cols)


In [ ]:
# Define the transformation functions using DataFrame input/output
scaler_X_func = lambda df: pd.DataFrame(scaler_X.transform(df[input_cols]), columns=input_cols)
scaler_y_func = lambda df: pd.DataFrame(scaler_y.transform(df[output_cols]), columns=output_cols)


In [ ]:
# Initialize scalers
scaler_X_vel = MinMaxScaler(feature_range=(-1, 1))


# Feature and target columns
input_cols = ['eta','eta_velocity']


# Extract training data as DataFrames
X_train = df_train_full[input_cols]


# Fit scalers
scaler_X_vel.fit(X_train)

# Define the transformation functions using DataFrame input/output
scaler_X_func_vel = lambda df: pd.DataFrame(scaler_X_vel.transform(df[input_cols]), columns=input_cols)



In [ ]:
# Initialize scalers
scaler_X_all = MinMaxScaler(feature_range=(-1, 1))


# Feature and target columns
input_cols = ['eta','eta_velocity','eta_acceleration']


# Extract training data as DataFrames
X_train = df_train_full[input_cols]


# Fit scalers
scaler_X_all.fit(X_train)

# Define the transformation functions using DataFrame input/output
scaler_X_func_all = lambda df: pd.DataFrame(scaler_X_all.transform(df[input_cols]), columns=input_cols)



In [ ]:
output_cols = ['heave']
# Initialize scalers
scaler_y_heave = MinMaxScaler(feature_range=(-1, 1))


scaler_y_heave.fit(df_train_full[output_cols])

# define scalling function

scaler_y_func_heave = lambda df: pd.DataFrame(scaler_y_heave.transform(df[output_cols]),columns=output_cols)

In [ ]:
output_cols = ['pitch']
# Initialize scalers

scaler_y_pitch = MinMaxScaler(feature_range=(-1, 1))

# Fit on training data (DataFrames, not NumPy arrays)

scaler_y_pitch.fit(df_train_full[output_cols])

# define scalling function

scaler_y_func_pitch = lambda df: pd.DataFrame(scaler_y_pitch.transform(df[output_cols]),columns=output_cols)

In [ ]:
output_cols = ['pendulum']
# Initialize scalers

scaler_y_pend = MinMaxScaler(feature_range=(-1, 1))

# Fit on training data (DataFrames, not NumPy arrays)

scaler_y_pend.fit(df_train_full[output_cols])

# define scalling function

scaler_y_func_pend = lambda df: pd.DataFrame(scaler_y_pend.transform(df[output_cols]),columns=output_cols)

# Building LSTM Model for the 3 dof

## $\eta$ only

In [ ]:
input_cols = ['eta']
output_cols = ['heave', 'pitch', 'pendulum']
nb= 5
nf=5
batch_size = 64


In [ ]:
# prepare training and validation data
zero_input = pd.DataFrame(np.zeros((5, len(input_cols))), columns=input_cols)
zero_output = pd.DataFrame(np.zeros((5, len(output_cols))), columns=output_cols)

input = df_case_train[input_cols]
output= df_case_train[output_cols]

input=pd.concat([zero_input, input], ignore_index=True)
output=pd.concat([zero_input, output], ignore_index=True)
# scale the data
input_df_scaled = scaler_X_func(input)
output_df_scaled = scaler_y_func(output)

X_train_seq,y_train_seq= create_exog_sequences(
    input_df = input_df_scaled,
    output_df=output_df_scaled,
    input_cols= input_cols,
    output_cols = output_cols,
    nb = nb,
    nf= nf
)

val_input = df_case_val[input_cols]
val_output= df_case_val[output_cols]
# scale the data
val_input_df_scaled = scaler_X_func(val_input)
val_output_df_scaled = scaler_y_func(val_output)

X_val_seq , Y_val_seq = create_exog_sequences(
    input_df = val_input_df_scaled,
    output_df= val_output_df_scaled,
    input_cols= input_cols,
    output_cols = output_cols,
    nb = nb,
    nf= nf
)

#Remove extra samples at the end to match batch size
def adjust_for_batch(X, y):
    n = (X.shape[0] // batch_size) * batch_size
    return X[:n], y[:n]

X_train_seq, y_train_seq = adjust_for_batch(X_train_seq, y_train_seq)
X_val_seq, Y_val_seq = adjust_for_batch(X_val_seq, Y_val_seq)

In [ ]:
# Set policy for mixed precision
policy = mixed_precision.Policy('float32')
mixed_precision.set_global_policy(policy)

#strategy = tf.distribute.MirroredStrategy()
strategy = tf.distribute.get_strategy()  # Use default strategy (single-device, CPU)

print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

In [ ]:
# Set environment variable to enable GPU memory growth
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Enable GPU (0) if you want to use GPU

# Now set memory growth for the GPU before initializing any device
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Check if the GPU is being used
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


In [ ]:
# Define the path where the model will be saved
checkpoint_path = "model_checkpoint_eta_only_nb5_nf5.h5"

# Set up the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    checkpoint_path,              # File path to save the model
    save_best_only=True,          # Save only the best model (based on the monitored metric)
    monitor='val_loss',           # Monitor validation loss to track the best model
    save_weights_only=False,       # Save only the model weights (you can change to False to save the whole model)
    verbose=1                     # Display a message when the model is saved
)
class ResetStatesCallback(Callback): # resert LSTM states at the end of each epoch
    def on_epoch_end(self, epoch, logs=None):
        self.model.reset_states()

# Define the EarlyStopping callback
early_stopping = EarlyStopping(
        monitor='val_loss',        # The metric to monitor, you can also use 'val_accuracy' or any other metric
        patience=10,               # Number of epochs with no improvement before stopping
        verbose=1,                 # Show a message when stopping
        restore_best_weights=True  # Restore model weights from the epoch with the best value of the monitored metric
    )



In [ ]:
    # --- Model Parameters ---

Lstm_1= 64

LSTM_2= 64
dropout = 0.2

# --- Create input shape info ---
time_steps = nb + 1 + nf  # full window
n_features = len(input_cols)  # e.g. 1


In [ ]:
# build model 

# --- Model input ---
seq_in = Input(batch_shape=(batch_size, time_steps, n_features), name="sequence")

# --- 1. Shared LSTM Layers ---
x = LSTM(Lstm_1, stateful=True, return_sequences=True, name='shared_lstm_1')(seq_in)
x = Dropout(dropout)(x)

x = LSTM(LSTM_2, stateful=True, return_sequences=False, name='shared_lstm_2')(x)
x = Dropout(dropout)(x)

# --- 2. Separate Dense Output Layers ---
heave_out = Dense(1, activation='tanh', name='heave')(x)
pitch_out = Dense(1, activation='tanh', name='pitch')(x)
pendulum_out = Dense(1, activation='tanh', name='pendulum')(x)

# --- 3. Define the model ---
model = Model(inputs=seq_in, outputs=[heave_out, pitch_out, pendulum_out])




In [ ]:
# complie model 
model.compile(
    optimizer='adam',
    loss={'heave':'mse','pitch':'mse','pendulum':'mse'},
    metrics={'heave':'mae','pitch':'mae','pendulum':'mae'}
)


In [ ]:
# Train model
history = model.fit(
    X_train_seq,
    {
        'heave': y_train_seq[:, 0],
        'pitch': y_train_seq[:, 1],
        'pendulum': y_train_seq[:, 2]
    },
    epochs=50,
    batch_size=batch_size,
    shuffle=False,
    validation_data=(
        X_val_seq,
        {
            'heave': Y_val_seq[:, 0],
            'pitch': Y_val_seq[:, 1],
            'pendulum': Y_val_seq[:, 2]
        }
    ),
    verbose=1,
    callbacks=[ResetStatesCallback(), early_stopping,checkpoint_callback]
)

In [ ]:

plt.figure(figsize=(12, 8))
history_dict = history.history
# Heave Loss
plt.subplot(3, 1, 1)
plt.plot(history_dict['heave_loss'], label='Training Heave Loss')
plt.plot(history_dict['val_heave_loss'], label='Validation Heave Loss')
plt.title('Heave Loss')
plt.xlabel('Epochs')
plt.ylabel('MSE [m²]')
plt.legend()

# Pitch Loss
plt.subplot(3, 1, 2)
plt.plot(history_dict['pitch_loss'], label='Training Pitch Loss')
plt.plot(history_dict['val_pitch_loss'], label='Validation Pitch Loss')
plt.title('Pitch Loss')
plt.xlabel('Epochs')
plt.ylabel('MSE [°²]')
plt.legend()

# Pendulum Loss
plt.subplot(3, 1, 3)
plt.plot(history_dict['pendulum_loss'], label='Training Pendulum Loss')
plt.plot(history_dict['val_pendulum_loss'], label='Validation Pendulum Loss')
plt.title('Pendulum Loss')
plt.xlabel('Epochs')
plt.ylabel('MSE [°²]')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
# Save dictionary to a .json file
with open('history_dict_new_1nd_run_eta_only_nb5_nf5.json', 'w') as f:
    json.dump(history_dict, f)

In [ ]:
# 1) Reset & warm-start as before
# This resets the states of the model, which is particularly important for stateful LSTMs.
model.reset_states()

# 2) Evaluate the model on the full validation set
test_results = model.evaluate(
    X_val_seq,  # The validation data
    {
        'heave': Y_val_seq[:, 0],   # True values for 'heave'
        'pitch': Y_val_seq[:, 1],   # True values for 'pitch'
        'pendulum': Y_val_seq[:, 2]  # True values for 'pendulum'
    },
    batch_size=batch_size,  # The batch size to use during evaluation
    verbose=1  # Verbosity level of evaluation
)

# Print the evaluation results
print("Validation results:", test_results)


In [ ]:
# ---- 1) Reset states before prediction ----
model.reset_states()

# ---- 2) Predict on training data ----
y_train_preds = model.predict(X_train_seq, batch_size=batch_size, verbose=1)
heave_pred, pitch_pred, pend_pred = [arr.squeeze(-1) for arr in y_train_preds]
y_train_pred_arr = np.vstack([heave_pred, pitch_pred, pend_pred]).T

# ---- 3) Predict on validation data ----
model.reset_states()
y_val_preds = model.predict(X_val_seq, batch_size=batch_size, verbose=1)
heave_pred, pitch_pred, pend_pred = [arr.squeeze(-1) for arr in y_val_preds]
y_val_pred_arr = np.vstack([heave_pred, pitch_pred, pend_pred]).T

# ---- 4) Create prediction DataFrames (scaled) ----
pred_cols = ['heave_pred', 'pitch_pred', 'pendulum_pred']
df_train_pred = pd.DataFrame(y_train_pred_arr, columns=pred_cols)
df_val_pred = pd.DataFrame(y_val_pred_arr, columns=pred_cols)

# ---- 5) Inverse transform predictions to original scale ----
train_pred_true_scale = scaler_y.inverse_transform(df_train_pred)
val_pred_true_scale = scaler_y.inverse_transform(df_val_pred)

# ---- 6) Create DataFrames for unscaled predictions ----
output_cols = ['heave', 'pitch', 'pendulum']
df_train_pred_true_scale = pd.DataFrame(train_pred_true_scale, columns=[f"{col}_pred" for col in output_cols])
df_val_pred_true_scale = pd.DataFrame(val_pred_true_scale, columns=[f"{col}_pred" for col in output_cols])

# ---- 7) Inverse transform ground truth Y data ----
y_train_true_scale = scaler_y.inverse_transform(y_train_seq)
y_val_true_scale = scaler_y.inverse_transform(Y_val_seq)

# ---- 8) Create DataFrames for ground truth ----
y_train_df_true_scale = pd.DataFrame(y_train_true_scale, columns=output_cols)
y_val_df_true_scale = pd.DataFrame(y_val_true_scale, columns=output_cols)

# ---- 9) Compute Metrics ----

# R² Scores
r2_scores_train = {
    col: r2_score(y_train_df_true_scale[col], df_train_pred_true_scale[f"{col}_pred"])
    for col in output_cols
}
r2_scores_val = {
    col: r2_score(y_val_df_true_scale[col], df_val_pred_true_scale[f"{col}_pred"])
    for col in output_cols
}

# RMSE
rmse_train = {
    col: np.sqrt(mean_squared_error(y_train_df_true_scale[col], df_train_pred_true_scale[f"{col}_pred"]))
    for col in output_cols
}
rmse_val = {
    col: np.sqrt(mean_squared_error(y_val_df_true_scale[col], df_val_pred_true_scale[f"{col}_pred"]))
    for col in output_cols
}

# ---- 10) Print Results ----
print("Training R² Scores:", r2_scores_train)
print("Validation R² Scores:", r2_scores_val)
print("Training RMSE:", rmse_train)
print("Validation RMSE:", rmse_val)


## $\eta$ and $\dot{\eta}$ 

In [ ]:
input_cols = ['eta','eta_velocity']
output_cols = ['heave', 'pitch', 'pendulum']
nb= 5
nf=5
batch_size = 64


In [ ]:
# prepare training and validation data
zero_input = pd.DataFrame(np.zeros((5, len(input_cols))), columns=input_cols)
zero_output = pd.DataFrame(np.zeros((5, len(output_cols))), columns=output_cols)

input = df_case_train[input_cols]
output= df_case_train[output_cols]

input=pd.concat([zero_input, input], ignore_index=True)
output=pd.concat([zero_input, output], ignore_index=True)

# scale the data
input_df_scaled = scaler_X_func_vel(input)
output_df_scaled = scaler_y_func(output)

X_train_seq,y_train_seq= create_exog_sequences(
    input_df = input_df_scaled,
    output_df=output_df_scaled,
    input_cols= input_cols,
    output_cols = output_cols,
    nb = nb,
    nf= nf
)

val_input = df_case_val[input_cols]
val_output= df_case_val[output_cols]
# scale the data
val_input_df_scaled = scaler_X_func_vel(val_input)
val_output_df_scaled = scaler_y_func(val_output)

X_val_seq , Y_val_seq = create_exog_sequences(
    input_df = val_input_df_scaled,
    output_df= val_output_df_scaled,
    input_cols= input_cols,
    output_cols = output_cols,
    nb = nb,
    nf= nf
)

#Remove extra samples at the end to match batch size
def adjust_for_batch(X, y):
    n = (X.shape[0] // batch_size) * batch_size
    return X[:n], y[:n]

X_train_seq, y_train_seq = adjust_for_batch(X_train_seq, y_train_seq)
X_val_seq, Y_val_seq = adjust_for_batch(X_val_seq, Y_val_seq)

In [ ]:
# Define the path where the model will be saved
checkpoint_path = "model_checkpoint_eta_Vel_nb5_nf5.h5"

# Set up the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    checkpoint_path,              # File path to save the model
    save_best_only=True,          # Save only the best model (based on the monitored metric)
    monitor='val_loss',           # Monitor validation loss to track the best model
    save_weights_only=False,       # Save only the model weights (you can change to False to save the whole model)
    verbose=1                     # Display a message when the model is saved
)

In [ ]:
    # --- Model Parameters ---

Lstm_1= 64

LSTM_2= 64
dropout = 0.2

#learning_rate = 1e-4

# --- Create input shape info ---
time_steps = nb + 1 + nf  # full window
n_features = len(input_cols)  # e.g. 1


In [ ]:
# build model 

# --- Model input ---
seq_in = Input(batch_shape=(batch_size, time_steps, n_features), name="sequence")

# --- 1. Shared LSTM Layers ---
x = LSTM(Lstm_1, stateful=True, return_sequences=True, name='shared_lstm_1')(seq_in)
x = Dropout(dropout)(x)

x = LSTM(LSTM_2, stateful=True, return_sequences=False, name='shared_lstm_2')(x)
x = Dropout(dropout)(x)

# --- 2. Separate Dense Output Layers ---
heave_out = Dense(1, activation='tanh', name='heave')(x)
pitch_out = Dense(1, activation='tanh', name='pitch')(x)
pendulum_out = Dense(1, activation='tanh', name='pendulum')(x)

# --- 3. Define the model ---
model = Model(inputs=seq_in, outputs=[heave_out, pitch_out, pendulum_out])




In [ ]:
# complie model 
model.compile(
    optimizer='adam',
    loss={'heave':'mse','pitch':'mse','pendulum':'mse'},
    metrics={'heave':'mae','pitch':'mae','pendulum':'mae'}
)


In [ ]:
# Train model
history = model.fit(
    X_train_seq,
    {
        'heave': y_train_seq[:, 0],
        'pitch': y_train_seq[:, 1],
        'pendulum': y_train_seq[:, 2]
    },
    epochs=50,
    batch_size=batch_size,
    shuffle=False,
    validation_data=(
        X_val_seq,
        {
            'heave': Y_val_seq[:, 0],
            'pitch': Y_val_seq[:, 1],
            'pendulum': Y_val_seq[:, 2]
        }
    ),
    verbose=1,
    callbacks=[ResetStatesCallback(), early_stopping,checkpoint_callback]
)

In [ ]:

plt.figure(figsize=(12, 8))
history_dict = history.history
# Heave Loss
plt.subplot(3, 1, 1)
plt.plot(history_dict['heave_loss'], label='Training Heave Loss')
plt.plot(history_dict['val_heave_loss'], label='Validation Heave Loss')
plt.title('Heave Loss')
plt.xlabel('Epochs')
plt.ylabel('MSE [m²]')
plt.legend()

# Pitch Loss
plt.subplot(3, 1, 2)
plt.plot(history_dict['pitch_loss'], label='Training Pitch Loss')
plt.plot(history_dict['val_pitch_loss'], label='Validation Pitch Loss')
plt.title('Pitch Loss')
plt.xlabel('Epochs')
plt.ylabel('MSE [°²]')
plt.legend()

# Pendulum Loss
plt.subplot(3, 1, 3)
plt.plot(history_dict['pendulum_loss'], label='Training Pendulum Loss')
plt.plot(history_dict['val_pendulum_loss'], label='Validation Pendulum Loss')
plt.title('Pendulum Loss')
plt.xlabel('Epochs')
plt.ylabel('MSE [°²]')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
import json

# Save dictionary to a .json file
with open('history_dict_new_1nd_run_eta_Vel_nb5_nf5.json', 'w') as f:
    json.dump(history_dict, f)

In [ ]:
# 1) Reset & warm-start as before
# This resets the states of the model, which is particularly important for stateful LSTMs.
model.reset_states()

# 2) Evaluate the model on the full validation set
test_results = model.evaluate(
    X_val_seq,  # The validation data
    {
        'heave': Y_val_seq[:, 0],   # True values for 'heave'
        'pitch': Y_val_seq[:, 1],   # True values for 'pitch'
        'pendulum': Y_val_seq[:, 2]  # True values for 'pendulum'
    },
    batch_size=batch_size,  # The batch size to use during evaluation
    verbose=1  # Verbosity level of evaluation
)

# Print the evaluation results
print("Validation results:", test_results)


In [ ]:
# ---- 1) Reset states before prediction ----
model.reset_states()

# ---- 2) Predict on training data ----
y_train_preds = model.predict(X_train_seq, batch_size=batch_size, verbose=1)
heave_pred, pitch_pred, pend_pred = [arr.squeeze(-1) for arr in y_train_preds]
y_train_pred_arr = np.vstack([heave_pred, pitch_pred, pend_pred]).T

# ---- 3) Predict on validation data ----
model.reset_states()
y_val_preds = model.predict(X_val_seq, batch_size=batch_size, verbose=1)
heave_pred, pitch_pred, pend_pred = [arr.squeeze(-1) for arr in y_val_preds]
y_val_pred_arr = np.vstack([heave_pred, pitch_pred, pend_pred]).T

# ---- 4) Create prediction DataFrames (scaled) ----
pred_cols = ['heave_pred', 'pitch_pred', 'pendulum_pred']
df_train_pred = pd.DataFrame(y_train_pred_arr, columns=pred_cols)
df_val_pred = pd.DataFrame(y_val_pred_arr, columns=pred_cols)

# ---- 5) Inverse transform predictions to original scale ----
train_pred_true_scale = scaler_y.inverse_transform(df_train_pred)
val_pred_true_scale = scaler_y.inverse_transform(df_val_pred)

# ---- 6) Create DataFrames for unscaled predictions ----
output_cols = ['heave', 'pitch', 'pendulum']
df_train_pred_true_scale = pd.DataFrame(train_pred_true_scale, columns=[f"{col}_pred" for col in output_cols])
df_val_pred_true_scale = pd.DataFrame(val_pred_true_scale, columns=[f"{col}_pred" for col in output_cols])

# ---- 7) Inverse transform ground truth Y data ----
y_train_true_scale = scaler_y.inverse_transform(y_train_seq)
y_val_true_scale = scaler_y.inverse_transform(Y_val_seq)

# ---- 8) Create DataFrames for ground truth ----
y_train_df_true_scale = pd.DataFrame(y_train_true_scale, columns=output_cols)
y_val_df_true_scale = pd.DataFrame(y_val_true_scale, columns=output_cols)

# ---- 9) Compute Metrics ----
from sklearn.metrics import r2_score, mean_squared_error

# R² Scores
r2_scores_train = {
    col: r2_score(y_train_df_true_scale[col], df_train_pred_true_scale[f"{col}_pred"])
    for col in output_cols
}
r2_scores_val = {
    col: r2_score(y_val_df_true_scale[col], df_val_pred_true_scale[f"{col}_pred"])
    for col in output_cols
}

# RMSE
rmse_train = {
    col: np.sqrt(mean_squared_error(y_train_df_true_scale[col], df_train_pred_true_scale[f"{col}_pred"]))
    for col in output_cols
}
rmse_val = {
    col: np.sqrt(mean_squared_error(y_val_df_true_scale[col], df_val_pred_true_scale[f"{col}_pred"]))
    for col in output_cols
}

# ---- 10) Print Results ----
print("Training R² Scores:", r2_scores_train)
print("Validation R² Scores:", r2_scores_val)
print("Training RMSE:", rmse_train)
print("Validation RMSE:", rmse_val)


## # $\eta$, $\dot{\eta}$, and $\ddot{\eta}$  

In [ ]:
input_cols = ['eta','eta_velocity','eta_acceleration']
output_cols = ['heave', 'pitch', 'pendulum']
nb= 5
nf=5
batch_size = 64


In [ ]:
# prepare training and validation data
zero_input = pd.DataFrame(np.zeros((5, len(input_cols))), columns=input_cols)
zero_output = pd.DataFrame(np.zeros((5, len(output_cols))), columns=output_cols)

input = df_case_train[input_cols]
output= df_case_train[output_cols]

input=pd.concat([zero_input, input], ignore_index=True)
output=pd.concat([zero_input, output], ignore_index=True)

# scale the data
input_df_scaled = scaler_X_func_all(input)
output_df_scaled = scaler_y_func(output)

X_train_seq,y_train_seq= create_exog_sequences(
    input_df = input_df_scaled,
    output_df=output_df_scaled,
    input_cols= input_cols,
    output_cols = output_cols,
    nb = nb,
    nf= nf
)

val_input = df_case_val[input_cols]
val_output= df_case_val[output_cols]
# scale the data
val_input_df_scaled = scaler_X_func_all(val_input)
val_output_df_scaled = scaler_y_func(val_output)

X_val_seq , Y_val_seq = create_exog_sequences(
    input_df = val_input_df_scaled,
    output_df= val_output_df_scaled,
    input_cols= input_cols,
    output_cols = output_cols,
    nb = nb,
    nf= nf
)

#Remove extra samples at the end to match batch size
def adjust_for_batch(X, y):
    n = (X.shape[0] // batch_size) * batch_size
    return X[:n], y[:n]

X_train_seq, y_train_seq = adjust_for_batch(X_train_seq, y_train_seq)
X_val_seq, Y_val_seq = adjust_for_batch(X_val_seq, Y_val_seq)

In [ ]:
# Define the path where the model will be saved
checkpoint_path = "model_checkpoint_eta_Vel_acc_nb5_nf5.h5"

# Set up the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    checkpoint_path,              # File path to save the model
    save_best_only=True,          # Save only the best model (based on the monitored metric)
    monitor='val_loss',           # Monitor validation loss to track the best model
    save_weights_only=False,       # Save only the model weights (you can change to False to save the whole model)
    verbose=1                     # Display a message when the model is saved
)

In [ ]:
    # --- Model Parameters ---

Lstm_1= 64

LSTM_2= 64
dropout = 0.2

#learning_rate = 1e-4

# --- Create input shape info ---
time_steps = nb + 1 + nf  # full window
n_features = len(input_cols)  # e.g. 1


In [ ]:
# build model 

# --- Model input ---
seq_in = Input(batch_shape=(batch_size, time_steps, n_features), name="sequence")

# --- 1. Shared LSTM Layers ---
x = LSTM(Lstm_1, stateful=True, return_sequences=True, name='shared_lstm_1')(seq_in)
x = Dropout(dropout)(x)

x = LSTM(LSTM_2, stateful=True, return_sequences=False, name='shared_lstm_2')(x)
x = Dropout(dropout)(x)

# --- 2. Separate Dense Output Layers ---
heave_out = Dense(1, activation='tanh', name='heave')(x)
pitch_out = Dense(1, activation='tanh', name='pitch')(x)
pendulum_out = Dense(1, activation='tanh', name='pendulum')(x)

# --- 3. Define the model ---
model = Model(inputs=seq_in, outputs=[heave_out, pitch_out, pendulum_out])




In [ ]:
# complie model 
model.compile(
    optimizer='adam',
    loss={'heave':'mse','pitch':'mse','pendulum':'mse'},
    metrics={'heave':'mae','pitch':'mae','pendulum':'mae'}
)


In [ ]:
# Train model
history = model.fit(
    X_train_seq,
    {
        'heave': y_train_seq[:, 0],
        'pitch': y_train_seq[:, 1],
        'pendulum': y_train_seq[:, 2]
    },
    epochs=50,
    batch_size=batch_size,
    shuffle=False,
    validation_data=(
        X_val_seq,
        {
            'heave': Y_val_seq[:, 0],
            'pitch': Y_val_seq[:, 1],
            'pendulum': Y_val_seq[:, 2]
        }
    ),
    verbose=1,
    callbacks=[ResetStatesCallback(), early_stopping,checkpoint_callback]
)

In [ ]:

plt.figure(figsize=(12, 8))
history_dict = history.history
# Heave Loss
plt.subplot(3, 1, 1)
plt.plot(history_dict['heave_loss'], label='Training Heave Loss')
plt.plot(history_dict['val_heave_loss'], label='Validation Heave Loss')
plt.title('Heave Loss')
plt.xlabel('Epochs')
plt.ylabel('MSE [m²]')
plt.legend()

# Pitch Loss
plt.subplot(3, 1, 2)
plt.plot(history_dict['pitch_loss'], label='Training Pitch Loss')
plt.plot(history_dict['val_pitch_loss'], label='Validation Pitch Loss')
plt.title('Pitch Loss')
plt.xlabel('Epochs')
plt.ylabel('MSE [°²]')
plt.legend()

# Pendulum Loss
plt.subplot(3, 1, 3)
plt.plot(history_dict['pendulum_loss'], label='Training Pendulum Loss')
plt.plot(history_dict['val_pendulum_loss'], label='Validation Pendulum Loss')
plt.title('Pendulum Loss')
plt.xlabel('Epochs')
plt.ylabel('MSE [°²]')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
import json

# Save dictionary to a .json file
with open('history_dict_new_1nd_run_eta_Vel_acc_nb5_nf5.json', 'w') as f:
    json.dump(history_dict, f)

In [ ]:
# 1) Reset & warm-start as before
# This resets the states of the model, which is particularly important for stateful LSTMs.
model.reset_states()

# 2) Evaluate the model on the full validation set
test_results = model.evaluate(
    X_val_seq,  # The validation data
    {
        'heave': Y_val_seq[:, 0],   # True values for 'heave'
        'pitch': Y_val_seq[:, 1],   # True values for 'pitch'
        'pendulum': Y_val_seq[:, 2]  # True values for 'pendulum'
    },
    batch_size=batch_size,  # The batch size to use during evaluation
    verbose=1  # Verbosity level of evaluation
)

# Print the evaluation results
print("Validation results:", test_results)


In [ ]:
# ---- 1) Reset states before prediction ----
model.reset_states()

# ---- 2) Predict on training data ----
y_train_preds = model.predict(X_train_seq, batch_size=batch_size, verbose=1)
heave_pred, pitch_pred, pend_pred = [arr.squeeze(-1) for arr in y_train_preds]
y_train_pred_arr = np.vstack([heave_pred, pitch_pred, pend_pred]).T

# ---- 3) Predict on validation data ----
model.reset_states()
y_val_preds = model.predict(X_val_seq, batch_size=batch_size, verbose=1)
heave_pred, pitch_pred, pend_pred = [arr.squeeze(-1) for arr in y_val_preds]
y_val_pred_arr = np.vstack([heave_pred, pitch_pred, pend_pred]).T

# ---- 4) Create prediction DataFrames (scaled) ----
pred_cols = ['heave_pred', 'pitch_pred', 'pendulum_pred']
df_train_pred = pd.DataFrame(y_train_pred_arr, columns=pred_cols)
df_val_pred = pd.DataFrame(y_val_pred_arr, columns=pred_cols)

# ---- 5) Inverse transform predictions to original scale ----
train_pred_true_scale = scaler_y.inverse_transform(df_train_pred)
val_pred_true_scale = scaler_y.inverse_transform(df_val_pred)

# ---- 6) Create DataFrames for unscaled predictions ----
output_cols = ['heave', 'pitch', 'pendulum']
df_train_pred_true_scale = pd.DataFrame(train_pred_true_scale, columns=[f"{col}_pred" for col in output_cols])
df_val_pred_true_scale = pd.DataFrame(val_pred_true_scale, columns=[f"{col}_pred" for col in output_cols])

# ---- 7) Inverse transform ground truth Y data ----
y_train_true_scale = scaler_y.inverse_transform(y_train_seq)
y_val_true_scale = scaler_y.inverse_transform(Y_val_seq)

# ---- 8) Create DataFrames for ground truth ----
y_train_df_true_scale = pd.DataFrame(y_train_true_scale, columns=output_cols)
y_val_df_true_scale = pd.DataFrame(y_val_true_scale, columns=output_cols)

# ---- 9) Compute Metrics ----
from sklearn.metrics import r2_score, mean_squared_error

# R² Scores
r2_scores_train = {
    col: r2_score(y_train_df_true_scale[col], df_train_pred_true_scale[f"{col}_pred"])
    for col in output_cols
}
r2_scores_val = {
    col: r2_score(y_val_df_true_scale[col], df_val_pred_true_scale[f"{col}_pred"])
    for col in output_cols
}

# RMSE
rmse_train = {
    col: np.sqrt(mean_squared_error(y_train_df_true_scale[col], df_train_pred_true_scale[f"{col}_pred"]))
    for col in output_cols
}
rmse_val = {
    col: np.sqrt(mean_squared_error(y_val_df_true_scale[col], df_val_pred_true_scale[f"{col}_pred"]))
    for col in output_cols
}

# ---- 10) Print Results ----
print("Training R² Scores:", r2_scores_train)
print("Validation R² Scores:", r2_scores_val)
print("Training RMSE:", rmse_train)
print("Validation RMSE:", rmse_val)


# Building a new model with 3 parralel layer followed by a shared layer

## $\eta$ and $\dot{\eta}$ 

In [ ]:
input_cols = ['eta','eta_velocity']
output_cols = ['heave', 'pitch', 'pendulum']
nb= 5
nf=5
batch_size = 64
    

In [ ]:
# prepare training and validation data
zero_input = pd.DataFrame(np.zeros((5, len(input_cols))), columns=input_cols)
zero_output = pd.DataFrame(np.zeros((5, len(output_cols))), columns=output_cols)

input = df_case_train[input_cols]
output= df_case_train[output_cols]

input=pd.concat([zero_input, input], ignore_index=True)
output=pd.concat([zero_input, output], ignore_index=True)

# scale the data
input_df_scaled = scaler_X_func_vel(input)
output_df_scaled = scaler_y_func(output)

X_train_seq,y_train_seq= create_exog_sequences(
    input_df = input_df_scaled,
    output_df=output_df_scaled,
    input_cols= input_cols,
    output_cols = output_cols,
    nb = nb,
    nf= nf
)

val_input = df_case_val[input_cols]
val_output= df_case_val[output_cols]
# scale the data
val_input_df_scaled = scaler_X_func_vel(val_input)
val_output_df_scaled = scaler_y_func(val_output)

X_val_seq , Y_val_seq = create_exog_sequences(
    input_df = val_input_df_scaled,
    output_df= val_output_df_scaled,
    input_cols= input_cols,
    output_cols = output_cols,
    nb = nb,
    nf= nf
)

#Remove extra samples at the end to match batch size
def adjust_for_batch(X, y):
    n = (X.shape[0] // batch_size) * batch_size
    return X[:n], y[:n]

X_train_seq, y_train_seq = adjust_for_batch(X_train_seq, y_train_seq)
X_val_seq, Y_val_seq = adjust_for_batch(X_val_seq, Y_val_seq)

In [ ]:

# Define the path where the model will be saved
checkpoint_path_3 = "model_checkpoint_eta_only_nb5_nf5_new_archt_eta_vel.h5"

# Set up the ModelCheckpoint callback
checkpoint_callback_3 = ModelCheckpoint(
    checkpoint_path_3,              # File path to save the model
    save_best_only=True,          # Save only the best model (based on the monitored metric)
    monitor='val_loss',           # Monitor validation loss to track the best model
    save_weights_only=False,       # Save only the model weights (you can change to False to save the whole model)
    verbose=1                     # Display a message when the model is saved
)

In [ ]:

# --- Model Parameters ---

heave_LSTM = 96
pitch_LSTM = 128
pend_LSTM = 64

shared_LSTM = 96
dropout = 0.2


# --- Create input shape info ---
time_steps = nb + 1 + nf  # full window
n_features = len(input_cols)  

    



In [ ]:
# build model 

seq_in = Input(batch_shape=(batch_size, time_steps, n_features), name="sequence")

# --- 1. Per-output LSTM layers ---
heave_branch = LSTM(heave_LSTM,stateful=True, return_sequences=True,  batch_size=batch_size, name='heave_lstm')(seq_in)
heave_branch = Dropout(dropout)(heave_branch)

pitch_branch = LSTM(pitch_LSTM,stateful=True, return_sequences=True,batch_size=batch_size,  name='pitch_lstm')(seq_in)
pitch_branch = Dropout(dropout)(pitch_branch)
    
pend_branch = LSTM(pend_LSTM, stateful=True, return_sequences=True, batch_size=batch_size, name='pendulum_lstm')(seq_in)
pend_branch = Dropout(dropout)(pend_branch)

# --- 2. Concatenate outputs ---
combined = Concatenate(name='combined_lstm_concat')([heave_branch, pitch_branch, pend_branch])

# --- 3. Shared LSTM layer to learn coupling ---
shared_lstm = LSTM(shared_LSTM, stateful=True,return_sequences=False,batch_size=batch_size,  name='shared_lstm')(combined)
shared_lstm = Dropout(dropout)(shared_lstm)

# --- 4. Final Dense outputs ---
heave_out = Dense(1, activation='tanh', name='heave')(shared_lstm)
pitch_out = Dense(1, activation='tanh', name='pitch')(shared_lstm)
pendulum_out = Dense(1, activation='tanh', name='pendulum')(shared_lstm)

# --- Define the model ---
model = Model(inputs=seq_in, outputs=[heave_out, pitch_out, pendulum_out])



In [ ]:
# complie model 
model.compile(
    optimizer='adam',
    loss={'heave':'mse','pitch':'mse','pendulum':'mse'},
    metrics={'heave':'mae','pitch':'mae','pendulum':'mae'}
)


In [ ]:
# Train model
history = model.fit(
    X_train_seq,
    {
        'heave': y_train_seq[:, 0],
        'pitch': y_train_seq[:, 1],
        'pendulum': y_train_seq[:, 2]
    },
    epochs=100,
    batch_size=batch_size,
    shuffle=False,
    validation_data=(
        X_val_seq,
        {
            'heave': Y_val_seq[:, 0],
            'pitch': Y_val_seq[:, 1],
            'pendulum': Y_val_seq[:, 2]
        }
    ),
    verbose=1,
    callbacks=[ResetStatesCallback(), early_stopping,checkpoint_callback_3]
)

In [ ]:

plt.figure(figsize=(12, 8))
history_dict = history.history
# Heave Loss
plt.subplot(3, 1, 1)
plt.plot(history_dict['heave_loss'], label='Training Heave Loss')
plt.plot(history_dict['val_heave_loss'], label='Validation Heave Loss')
plt.title('Heave Loss')
plt.xlabel('Epochs')
plt.ylabel('MSE [m²]')
plt.legend()

# Pitch Loss
plt.subplot(3, 1, 2)
plt.plot(history_dict['pitch_loss'], label='Training Pitch Loss')
plt.plot(history_dict['val_pitch_loss'], label='Validation Pitch Loss')
plt.title('Pitch Loss')
plt.xlabel('Epochs')
plt.ylabel('MSE [°²]')
plt.legend()

# Pendulum Loss
plt.subplot(3, 1, 3)
plt.plot(history_dict['pendulum_loss'], label='Training Pendulum Loss')
plt.plot(history_dict['val_pendulum_loss'], label='Validation Pendulum Loss')
plt.title('Pendulum Loss')
plt.xlabel('Epochs')
plt.ylabel('MSE [°²]')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
import json

# Save dictionary to a .json file
with open('history_dict_new_1nd_run_eta_vel_nb5_nf5_new_archt_new.json', 'w') as f:
    json.dump(history_dict, f)

In [ ]:
# 1) Reset & warm-start as before
# This resets the states of the model, which is particularly important for stateful LSTMs.
model.reset_states()

# 2) Evaluate the model on the full validation set
test_results = model.evaluate(
    X_val_seq,  # The validation data
    {
        'heave': Y_val_seq[:, 0],   # True values for 'heave'
        'pitch': Y_val_seq[:, 1],   # True values for 'pitch'
        'pendulum': Y_val_seq[:, 2]  # True values for 'pendulum'
    },
    batch_size=batch_size,  # The batch size to use during evaluation
    verbose=1  # Verbosity level of evaluation
)

# Print the evaluation results
print("Validation results:", test_results)


In [ ]:
from tensorflow.keras.models import load_model

model = load_model("model_checkpoint_eta_only_nb5_nf5_new_archt_eta_vel.h5")


In [ ]:
# ---- 1) Reset states before prediction ----
model.reset_states()

# ---- 2) Predict on training data ----
y_train_preds = model.predict(X_train_seq, batch_size=batch_size, verbose=1)
heave_pred, pitch_pred, pend_pred = [arr.squeeze(-1) for arr in y_train_preds]
y_train_pred_arr = np.vstack([heave_pred, pitch_pred, pend_pred]).T

# ---- 3) Predict on validation data ----
model.reset_states()
y_val_preds = model.predict(X_val_seq, batch_size=batch_size, verbose=1)
heave_pred, pitch_pred, pend_pred = [arr.squeeze(-1) for arr in y_val_preds]
y_val_pred_arr = np.vstack([heave_pred, pitch_pred, pend_pred]).T

# ---- 4) Create prediction DataFrames (scaled) ----
pred_cols = ['heave_pred', 'pitch_pred', 'pendulum_pred']
df_train_pred = pd.DataFrame(y_train_pred_arr, columns=pred_cols)
df_val_pred = pd.DataFrame(y_val_pred_arr, columns=pred_cols)

# ---- 5) Inverse transform predictions to original scale ----
train_pred_true_scale = scaler_y.inverse_transform(df_train_pred)
val_pred_true_scale = scaler_y.inverse_transform(df_val_pred)

# ---- 6) Create DataFrames for unscaled predictions ----
output_cols = ['heave', 'pitch', 'pendulum']
df_train_pred_true_scale = pd.DataFrame(train_pred_true_scale, columns=[f"{col}_pred" for col in output_cols])
df_val_pred_true_scale = pd.DataFrame(val_pred_true_scale, columns=[f"{col}_pred" for col in output_cols])

# ---- 7) Inverse transform ground truth Y data ----
y_train_true_scale = scaler_y.inverse_transform(y_train_seq)
y_val_true_scale = scaler_y.inverse_transform(Y_val_seq)

# ---- 8) Create DataFrames for ground truth ----
y_train_df_true_scale = pd.DataFrame(y_train_true_scale, columns=output_cols)
y_val_df_true_scale = pd.DataFrame(y_val_true_scale, columns=output_cols)

# ---- 9) Compute Metrics ----
from sklearn.metrics import r2_score, mean_squared_error

# R² Scores
r2_scores_train = {
    col: r2_score(y_train_df_true_scale[col], df_train_pred_true_scale[f"{col}_pred"])
    for col in output_cols
}
r2_scores_val = {
    col: r2_score(y_val_df_true_scale[col], df_val_pred_true_scale[f"{col}_pred"])
    for col in output_cols
}

# RMSE
rmse_train = {
    col: np.sqrt(mean_squared_error(y_train_df_true_scale[col], df_train_pred_true_scale[f"{col}_pred"]))
    for col in output_cols
}
rmse_val = {
    col: np.sqrt(mean_squared_error(y_val_df_true_scale[col], df_val_pred_true_scale[f"{col}_pred"]))
    for col in output_cols
}

# ---- 10) Print Results ----
print("Training R² Scores:", r2_scores_train)
print("Validation R² Scores:", r2_scores_val)
print("Training RMSE:", rmse_train)
print("Validation RMSE:", rmse_val)


## $\eta$, $\dot{\eta}$ , and $\ddot{\eta}$ 

In [ ]:
input_cols = ['eta','eta_velocity','eta_acceleration']
output_cols = ['heave', 'pitch', 'pendulum']
nb= 5
nf=5
batch_size = 64
    

In [ ]:
# prepare training and validation data
zero_input = pd.DataFrame(np.zeros((5, len(input_cols))), columns=input_cols)
zero_output = pd.DataFrame(np.zeros((5, len(output_cols))), columns=output_cols)

input = df_case_train[input_cols]
output= df_case_train[output_cols]

input=pd.concat([zero_input, input], ignore_index=True)
output=pd.concat([zero_input, output], ignore_index=True)
# scale the data
input_df_scaled = scaler_X_func_all(input)
output_df_scaled = scaler_y_func(output)

X_train_seq,y_train_seq= create_exog_sequences(
    input_df = input_df_scaled,
    output_df=output_df_scaled,
    input_cols= input_cols,
    output_cols = output_cols,
    nb = nb,
    nf= nf
)

val_input = df_case_val[input_cols]
val_output= df_case_val[output_cols]
# scale the data
val_input_df_scaled = scaler_X_func_all(val_input)
val_output_df_scaled = scaler_y_func(val_output)

X_val_seq , Y_val_seq = create_exog_sequences(
    input_df = val_input_df_scaled,
    output_df= val_output_df_scaled,
    input_cols= input_cols,
    output_cols = output_cols,
    nb = nb,
    nf= nf
)

#Remove extra samples at the end to match batch size
def adjust_for_batch(X, y):
    n = (X.shape[0] // batch_size) * batch_size
    return X[:n], y[:n]

X_train_seq, y_train_seq = adjust_for_batch(X_train_seq, y_train_seq)
X_val_seq, Y_val_seq = adjust_for_batch(X_val_seq, Y_val_seq)

In [ ]:

# Define the path where the model will be saved
checkpoint_path_3 = "model_checkpoint_eta_only_nb5_nf5_new_archt_eta_vel_acc.h5"

# Set up the ModelCheckpoint callback
checkpoint_callback_3 = ModelCheckpoint(
    checkpoint_path_3,              # File path to save the model
    save_best_only=True,          # Save only the best model (based on the monitored metric)
    monitor='val_loss',           # Monitor validation loss to track the best model
    save_weights_only=False,       # Save only the model weights (you can change to False to save the whole model)
    verbose=1                     # Display a message when the model is saved
)

In [ ]:

# --- Model Parameters ---

heave_LSTM = 64
pitch_LSTM = 64
pend_LSTM = 64

shared_LSTM = 64
dropout = 0.2


# --- Create input shape info ---
time_steps = nb + 1 + nf  # full window
n_features = len(input_cols)  

    



In [ ]:
# build model 

seq_in = Input(batch_shape=(batch_size, time_steps, n_features), name="sequence")

# --- 1. Per-output LSTM layers ---
heave_branch = LSTM(heave_LSTM,stateful=True, return_sequences=True,  batch_size=batch_size, name='heave_lstm')(seq_in)
heave_branch = Dropout(dropout)(heave_branch)

pitch_branch = LSTM(pitch_LSTM,stateful=True, return_sequences=True,batch_size=batch_size,  name='pitch_lstm')(seq_in)
pitch_branch = Dropout(dropout)(pitch_branch)
    
pend_branch = LSTM(pend_LSTM, stateful=True, return_sequences=True, batch_size=batch_size, name='pendulum_lstm')(seq_in)
pend_branch = Dropout(dropout)(pend_branch)

# --- 2. Concatenate outputs ---
combined = Concatenate(name='combined_lstm_concat')([heave_branch, pitch_branch, pend_branch])

# --- 3. Shared LSTM layer to learn coupling ---
shared_lstm = LSTM(shared_LSTM, stateful=True,return_sequences=False,batch_size=batch_size,  name='shared_lstm')(combined)
shared_lstm = Dropout(dropout)(shared_lstm)

# --- 4. Final Dense outputs ---
heave_out = Dense(1, activation='tanh', name='heave')(shared_lstm)
pitch_out = Dense(1, activation='tanh', name='pitch')(shared_lstm)
pendulum_out = Dense(1, activation='tanh', name='pendulum')(shared_lstm)

# --- Define the model ---
model = Model(inputs=seq_in, outputs=[heave_out, pitch_out, pendulum_out])



In [ ]:
# complie model 
model.compile(
    optimizer='adam',
    loss={'heave':'mse','pitch':'mse','pendulum':'mse'},
    metrics={'heave':'mae','pitch':'mae','pendulum':'mae'}
)


In [ ]:
# Train model
history = model.fit(
    X_train_seq,
    {
        'heave': y_train_seq[:, 0],
        'pitch': y_train_seq[:, 1],
        'pendulum': y_train_seq[:, 2]
    },
    epochs=100,
    batch_size=batch_size,
    shuffle=False,
    validation_data=(
        X_val_seq,
        {
            'heave': Y_val_seq[:, 0],
            'pitch': Y_val_seq[:, 1],
            'pendulum': Y_val_seq[:, 2]
        }
    ),
    verbose=1,
    callbacks=[ResetStatesCallback(), early_stopping,checkpoint_callback_3]
)

In [ ]:

plt.figure(figsize=(12, 8))
history_dict = history.history
# Heave Loss
plt.subplot(3, 1, 1)
plt.plot(history_dict['heave_loss'], label='Training Heave Loss')
plt.plot(history_dict['val_heave_loss'], label='Validation Heave Loss')
plt.title('Heave Loss')
plt.xlabel('Epochs')
plt.ylabel('MSE [m²]')
plt.legend()

# Pitch Loss
plt.subplot(3, 1, 2)
plt.plot(history_dict['pitch_loss'], label='Training Pitch Loss')
plt.plot(history_dict['val_pitch_loss'], label='Validation Pitch Loss')
plt.title('Pitch Loss')
plt.xlabel('Epochs')
plt.ylabel('MSE [°²]')
plt.legend()

# Pendulum Loss
plt.subplot(3, 1, 3)
plt.plot(history_dict['pendulum_loss'], label='Training Pendulum Loss')
plt.plot(history_dict['val_pendulum_loss'], label='Validation Pendulum Loss')
plt.title('Pendulum Loss')
plt.xlabel('Epochs')
plt.ylabel('MSE [°²]')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
import json

# Save dictionary to a .json file
with open('history_dict_new_1nd_run_eta_vel_acc_nb5_nf5_new_archt_new.json', 'w') as f:
    json.dump(history_dict, f)

In [ ]:
# 1) Reset & warm-start as before
# This resets the states of the model, which is particularly important for stateful LSTMs.
model.reset_states()

# 2) Evaluate the model on the full validation set
test_results = model.evaluate(
    X_val_seq,  # The validation data
    {
        'heave': Y_val_seq[:, 0],   # True values for 'heave'
        'pitch': Y_val_seq[:, 1],   # True values for 'pitch'
        'pendulum': Y_val_seq[:, 2]  # True values for 'pendulum'
    },
    batch_size=batch_size,  # The batch size to use during evaluation
    verbose=1  # Verbosity level of evaluation
)

# Print the evaluation results
print("Validation results:", test_results)


In [ ]:

model = load_model("model_checkpoint_eta_only_nb5_nf5_new_archt_eta_vel_acc.h5")


In [ ]:
# ---- 1) Reset states before prediction ----
model.reset_states()

# ---- 2) Predict on training data ----
y_train_preds = model.predict(X_train_seq, batch_size=batch_size, verbose=1)
heave_pred, pitch_pred, pend_pred = [arr.squeeze(-1) for arr in y_train_preds]
y_train_pred_arr = np.vstack([heave_pred, pitch_pred, pend_pred]).T

# ---- 3) Predict on validation data ----
model.reset_states()
y_val_preds = model.predict(X_val_seq, batch_size=batch_size, verbose=1)
heave_pred, pitch_pred, pend_pred = [arr.squeeze(-1) for arr in y_val_preds]
y_val_pred_arr = np.vstack([heave_pred, pitch_pred, pend_pred]).T

# ---- 4) Create prediction DataFrames (scaled) ----
pred_cols = ['heave_pred', 'pitch_pred', 'pendulum_pred']
df_train_pred = pd.DataFrame(y_train_pred_arr, columns=pred_cols)
df_val_pred = pd.DataFrame(y_val_pred_arr, columns=pred_cols)

# ---- 5) Inverse transform predictions to original scale ----
train_pred_true_scale = scaler_y.inverse_transform(df_train_pred)
val_pred_true_scale = scaler_y.inverse_transform(df_val_pred)

# ---- 6) Create DataFrames for unscaled predictions ----
output_cols = ['heave', 'pitch', 'pendulum']
df_train_pred_true_scale = pd.DataFrame(train_pred_true_scale, columns=[f"{col}_pred" for col in output_cols])
df_val_pred_true_scale = pd.DataFrame(val_pred_true_scale, columns=[f"{col}_pred" for col in output_cols])

# ---- 7) Inverse transform ground truth Y data ----
y_train_true_scale = scaler_y.inverse_transform(y_train_seq)
y_val_true_scale = scaler_y.inverse_transform(Y_val_seq)

# ---- 8) Create DataFrames for ground truth ----
y_train_df_true_scale = pd.DataFrame(y_train_true_scale, columns=output_cols)
y_val_df_true_scale = pd.DataFrame(y_val_true_scale, columns=output_cols)

# ---- 9) Compute Metrics ----

# R² Scores
r2_scores_train = {
    col: r2_score(y_train_df_true_scale[col], df_train_pred_true_scale[f"{col}_pred"])
    for col in output_cols
}
r2_scores_val = {
    col: r2_score(y_val_df_true_scale[col], df_val_pred_true_scale[f"{col}_pred"])
    for col in output_cols
}

# RMSE
rmse_train = {
    col: np.sqrt(mean_squared_error(y_train_df_true_scale[col], df_train_pred_true_scale[f"{col}_pred"]))
    for col in output_cols
}
rmse_val = {
    col: np.sqrt(mean_squared_error(y_val_df_true_scale[col], df_val_pred_true_scale[f"{col}_pred"]))
    for col in output_cols
}

# ---- 10) Print Results ----
print("Training R² Scores:", r2_scores_train)
print("Validation R² Scores:", r2_scores_val)
print("Training RMSE:", rmse_train)
print("Validation RMSE:", rmse_val)


### Adjusting stopping creteria only

In [ ]:
model = load_model("model_checkpoint_eta_only_nb5_nf5_new_archt_eta_vel_acc.h5")


In [ ]:

# Define the path where the model will be saved
checkpoint_path_3 = "model_checkpoint_eta_only_nb5_nf5_new_archt_eta_vel_acc_more_stopping_creteria.h5"

# Set up the ModelCheckpoint callback
checkpoint_callback_3 = ModelCheckpoint(
    checkpoint_path_3,              # File path to save the model
    save_best_only=True,          # Save only the best model (based on the monitored metric)
    monitor='val_loss',           # Monitor validation loss to track the best model
    save_weights_only=False,       # Save only the model weights (you can change to False to save the whole model)
    verbose=1                     # Display a message when the model is saved
)

In [ ]:
# Define the EarlyStopping callback
early_stopping_more = EarlyStopping(
        monitor='val_loss',        # The metric to monitor, you can also use 'val_accuracy' or any other metric
        patience=15,               # Number of epochs with no improvement before stopping
        verbose=1,                 # Show a message when stopping
        restore_best_weights=True  # Restore model weights from the epoch with the best value of the monitored metric
    )


In [ ]:
# complie model 
model.compile(
    optimizer='adam',
    loss={'heave':'mse','pitch':'mse','pendulum':'mse'},
    metrics={'heave':'mae','pitch':'mae','pendulum':'mae'}
)


In [ ]:
# Train model
history = model.fit(
    X_train_seq,
    {
        'heave': y_train_seq[:, 0],
        'pitch': y_train_seq[:, 1],
        'pendulum': y_train_seq[:, 2]
    },
    epochs=200,
    batch_size=batch_size,
    shuffle=False,
    validation_data=(
        X_val_seq,
        {
            'heave': Y_val_seq[:, 0],
            'pitch': Y_val_seq[:, 1],
            'pendulum': Y_val_seq[:, 2]
        }
    ),
    verbose=1,
    callbacks=[ResetStatesCallback(), early_stopping_more,checkpoint_callback_3]
)

In [ ]:

plt.figure(figsize=(12, 8))
history_dict = history.history
# Heave Loss
plt.subplot(3, 1, 1)
plt.plot(history_dict['heave_loss'], label='Training Heave Loss')
plt.plot(history_dict['val_heave_loss'], label='Validation Heave Loss')
plt.title('Heave Loss')
plt.xlabel('Epochs')
plt.ylabel('MSE [m²]')
plt.legend()

# Pitch Loss
plt.subplot(3, 1, 2)
plt.plot(history_dict['pitch_loss'], label='Training Pitch Loss')
plt.plot(history_dict['val_pitch_loss'], label='Validation Pitch Loss')
plt.title('Pitch Loss')
plt.xlabel('Epochs')
plt.ylabel('MSE [°²]')
plt.legend()

# Pendulum Loss
plt.subplot(3, 1, 3)
plt.plot(history_dict['pendulum_loss'], label='Training Pendulum Loss')
plt.plot(history_dict['val_pendulum_loss'], label='Validation Pendulum Loss')
plt.title('Pendulum Loss')
plt.xlabel('Epochs')
plt.ylabel('MSE [°²]')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
# Save dictionary to a .json file
with open('history_dict_new_1nd_run_eta_vel_acc_nb5_nf5_new_archt_new_mor_stopping.json', 'w') as f:
    json.dump(history_dict, f)

In [ ]:
# 1) Reset & warm-start as before
# This resets the states of the model, which is particularly important for stateful LSTMs.
model.reset_states()

# 2) Evaluate the model on the full validation set
test_results = model.evaluate(
    X_val_seq,  # The validation data
    {
        'heave': Y_val_seq[:, 0],   # True values for 'heave'
        'pitch': Y_val_seq[:, 1],   # True values for 'pitch'
        'pendulum': Y_val_seq[:, 2]  # True values for 'pendulum'
    },
    batch_size=batch_size,  # The batch size to use during evaluation
    verbose=1  # Verbosity level of evaluation
)

# Print the evaluation results
print("Validation results:", test_results)


In [ ]:
model = load_model("model_checkpoint_eta_only_nb5_nf5_new_archt_eta_vel_acc_more_stopping_creteria.h5")


In [ ]:
# ---- 1) Reset states before prediction ----
model.reset_states()

# ---- 2) Predict on training data ----
y_train_preds = model.predict(X_train_seq, batch_size=batch_size, verbose=1)
heave_pred, pitch_pred, pend_pred = [arr.squeeze(-1) for arr in y_train_preds]
y_train_pred_arr = np.vstack([heave_pred, pitch_pred, pend_pred]).T

# ---- 3) Predict on validation data ----
model.reset_states()
y_val_preds = model.predict(X_val_seq, batch_size=batch_size, verbose=1)
heave_pred, pitch_pred, pend_pred = [arr.squeeze(-1) for arr in y_val_preds]
y_val_pred_arr = np.vstack([heave_pred, pitch_pred, pend_pred]).T

# ---- 4) Create prediction DataFrames (scaled) ----
pred_cols = ['heave_pred', 'pitch_pred', 'pendulum_pred']
df_train_pred = pd.DataFrame(y_train_pred_arr, columns=pred_cols)
df_val_pred = pd.DataFrame(y_val_pred_arr, columns=pred_cols)

# ---- 5) Inverse transform predictions to original scale ----
train_pred_true_scale = scaler_y.inverse_transform(df_train_pred)
val_pred_true_scale = scaler_y.inverse_transform(df_val_pred)

# ---- 6) Create DataFrames for unscaled predictions ----
output_cols = ['heave', 'pitch', 'pendulum']
df_train_pred_true_scale = pd.DataFrame(train_pred_true_scale, columns=[f"{col}_pred" for col in output_cols])
df_val_pred_true_scale = pd.DataFrame(val_pred_true_scale, columns=[f"{col}_pred" for col in output_cols])

# ---- 7) Inverse transform ground truth Y data ----
y_train_true_scale = scaler_y.inverse_transform(y_train_seq)
y_val_true_scale = scaler_y.inverse_transform(Y_val_seq)

# ---- 8) Create DataFrames for ground truth ----
y_train_df_true_scale = pd.DataFrame(y_train_true_scale, columns=output_cols)
y_val_df_true_scale = pd.DataFrame(y_val_true_scale, columns=output_cols)

# ---- 9) Compute Metrics ----

# R² Scores
r2_scores_train = {
    col: r2_score(y_train_df_true_scale[col], df_train_pred_true_scale[f"{col}_pred"])
    for col in output_cols
}
r2_scores_val = {
    col: r2_score(y_val_df_true_scale[col], df_val_pred_true_scale[f"{col}_pred"])
    for col in output_cols
}

# RMSE
rmse_train = {
    col: np.sqrt(mean_squared_error(y_train_df_true_scale[col], df_train_pred_true_scale[f"{col}_pred"]))
    for col in output_cols
}
rmse_val = {
    col: np.sqrt(mean_squared_error(y_val_df_true_scale[col], df_val_pred_true_scale[f"{col}_pred"]))
    for col in output_cols
}

# ---- 10) Print Results ----
print("Training R² Scores:", r2_scores_train)
print("Validation R² Scores:", r2_scores_val)
print("Training RMSE:", rmse_train)
print("Validation RMSE:", rmse_val)


In [ ]:
y_train_df_true_scale.to_csv('LSTM_train_true.csv')
df_train_pred_true_scale.to_csv('LSTM_train_pred.csv')

y_val_df_true_scale.to_csv('LSTM_val_true.csv')
df_val_pred_true_scale.to_csv('LSTM_val_pred.csv')

In [ ]:
# Combine histories from two runs into one dictionary
# Load history dictionary from JSON file
with open('history_dict_new_1nd_run_eta_vel_acc_nb5_nf5_new_archt_new.json', 'r') as f:
    history_dict = json.load(f)
with open('history_dict_new_1nd_run_eta_vel_acc_nb5_nf5_new_archt_new_mor_stopping.json', 'r') as f:
    history_dict_1 = json.load(f)
                               # Merge histories 
history_combined = {}

# List of history dicts
history_list = [history_dict,history_dict_1]

# Keys to merge (assuming all histories share the same keys)
keys = history_dict.keys()

# Combine each key manually
for key in keys:
    history_combined[key] = history_dict.get(key, []) + \
                            history_dict_1.get(key, []) 

In [ ]:
# Heave Loss Plot
plt.figure(figsize=(10, 4))
plt.plot(history_combined['heave_loss'], label='Training Heave Loss')
plt.plot(history_combined['val_heave_loss'], label='Validation Heave Loss')
plt.title('Heave Loss')
plt.xlabel('Epochs')
plt.ylabel('MSE [m²]')
plt.legend()
plt.tight_layout()
plt.savefig("heave_loss.png")
plt.show()

# Pitch Loss Plot
plt.figure(figsize=(10, 4))
plt.plot(history_combined['pitch_loss'], label='Training Pitch Loss')
plt.plot(history_combined['val_pitch_loss'], label='Validation Pitch Loss')
plt.title('Pitch Loss')
plt.xlabel('Epochs')
plt.ylabel('MSE [°²]')
plt.legend()
plt.tight_layout()
plt.savefig("pitch_loss.png")
plt.show()

# Pendulum Loss Plot
plt.figure(figsize=(10, 4))
plt.plot(history_combined['pendulum_loss'], label='Training Pendulum Loss')
plt.plot(history_combined['val_pendulum_loss'], label='Validation Pendulum Loss')
plt.title('Pendulum Loss')
plt.xlabel('Epochs')
plt.ylabel('MSE [°²]')
plt.legend()
plt.tight_layout()
plt.savefig("pendulum_loss.png")
plt.show()


### Adding more units for pitch layer and adjusting stopping creteria

In [ ]:
# --- Model Parameters ---

heave_LSTM = 64
pitch_LSTM = 96
pend_LSTM = 64

shared_LSTM = 64
dropout = 0.2


# --- Create input shape info ---
time_steps = nb + 1 + nf  # full window
n_features = len(input_cols)  # e.g. 1

    



In [ ]:


# Input layer
seq_in = Input(batch_shape=(batch_size, time_steps, n_features), name="sequence")

# --- 1. Shared LSTM layer to extract shared temporal features ---
shared_lstm = LSTM(shared_LSTM, stateful=True, return_sequences=True, name='shared_lstm')(seq_in)
shared_lstm = Dropout(dropout)(shared_lstm)

# --- 2. Per-output LSTM branches (individual DoF refinement) ---
heave_branch = LSTM(heave_LSTM, stateful=True, return_sequences=False, name='heave_lstm')(shared_lstm)
heave_branch = Dropout(dropout)(heave_branch)

pitch_branch = LSTM(pitch_LSTM, stateful=True, return_sequences=False, name='pitch_lstm')(shared_lstm)
pitch_branch = Dropout(dropout)(pitch_branch)

pend_branch = LSTM(pend_LSTM, stateful=True, return_sequences=False, name='pendulum_lstm')(shared_lstm)
pend_branch = Dropout(dropout)(pend_branch)

# --- 3. Final Dense outputs ---
heave_out = Dense(1, activation='tanh', name='heave')(heave_branch)
pitch_out = Dense(1, activation='tanh', name='pitch')(pitch_branch)
pendulum_out = Dense(1, activation='tanh', name='pendulum')(pend_branch)

# --- 4. Define the model ---
model = Model(inputs=seq_in, outputs=[heave_out, pitch_out, pendulum_out])


In [ ]:
# complie model 
model.compile(
    optimizer='adam',
    loss={'heave':'mse','pitch':'mse','pendulum':'mse'},
    metrics={'heave':'mae','pitch':'mae','pendulum':'mae'}
)


In [ ]:
# Train model
history = model.fit(
    X_train_seq,
    {
        'heave': y_train_seq[:, 0],
        'pitch': y_train_seq[:, 1],
        'pendulum': y_train_seq[:, 2]
    },
    epochs=200,
    batch_size=batch_size,
    shuffle=False,
    validation_data=(
        X_val_seq,
        {
            'heave': Y_val_seq[:, 0],
            'pitch': Y_val_seq[:, 1],
            'pendulum': Y_val_seq[:, 2]
        }
    ),
    verbose=1,
    callbacks=[ResetStatesCallback(), early_stopping_more,checkpoint_callback_3]
)

In [ ]:

plt.figure(figsize=(12, 8))
history_dict = history.history
# Heave Loss
plt.subplot(3, 1, 1)
plt.plot(history_dict['heave_loss'], label='Training Heave Loss')
plt.plot(history_dict['val_heave_loss'], label='Validation Heave Loss')
plt.title('Heave Loss')
plt.xlabel('Epochs')
plt.ylabel('MSE [m²]')
plt.legend()

# Pitch Loss
plt.subplot(3, 1, 2)
plt.plot(history_dict['pitch_loss'], label='Training Pitch Loss')
plt.plot(history_dict['val_pitch_loss'], label='Validation Pitch Loss')
plt.title('Pitch Loss')
plt.xlabel('Epochs')
plt.ylabel('MSE [°²]')
plt.legend()

# Pendulum Loss
plt.subplot(3, 1, 3)
plt.plot(history_dict['pendulum_loss'], label='Training Pendulum Loss')
plt.plot(history_dict['val_pendulum_loss'], label='Validation Pendulum Loss')
plt.title('Pendulum Loss')
plt.xlabel('Epochs')
plt.ylabel('MSE [°²]')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
import json

# Save dictionary to a .json file
with open('history_dict_new_1nd_run_eta_vel_acc_nb5_nf5_new_archt_new_more_units.json', 'w') as f:
    json.dump(history_dict, f)

In [ ]:
# 1) Reset & warm-start as before
# This resets the states of the model, which is particularly important for stateful LSTMs.
model.reset_states()

# 2) Evaluate the model on the full validation set
test_results = model.evaluate(
    X_val_seq,  # The validation data
    {
        'heave': Y_val_seq[:, 0],   # True values for 'heave'
        'pitch': Y_val_seq[:, 1],   # True values for 'pitch'
        'pendulum': Y_val_seq[:, 2]  # True values for 'pendulum'
    },
    batch_size=batch_size,  # The batch size to use during evaluation
    verbose=1  # Verbosity level of evaluation
)

# Print the evaluation results
print("Validation results:", test_results)


In [ ]:
model = load_model("model_checkpoint_eta_only_nb5_nf5_new_archt_eta_vel_acc_more_units.h5")


In [ ]:
# ---- 1) Reset states before prediction ----
model.reset_states()

# ---- 2) Predict on training data ----
y_train_preds = model.predict(X_train_seq, batch_size=batch_size, verbose=1)
heave_pred, pitch_pred, pend_pred = [arr.squeeze(-1) for arr in y_train_preds]
y_train_pred_arr = np.vstack([heave_pred, pitch_pred, pend_pred]).T

# ---- 3) Predict on validation data ----
model.reset_states()
y_val_preds = model.predict(X_val_seq, batch_size=batch_size, verbose=1)
heave_pred, pitch_pred, pend_pred = [arr.squeeze(-1) for arr in y_val_preds]
y_val_pred_arr = np.vstack([heave_pred, pitch_pred, pend_pred]).T

# ---- 4) Create prediction DataFrames (scaled) ----
pred_cols = ['heave_pred', 'pitch_pred', 'pendulum_pred']
df_train_pred = pd.DataFrame(y_train_pred_arr, columns=pred_cols)
df_val_pred = pd.DataFrame(y_val_pred_arr, columns=pred_cols)

# ---- 5) Inverse transform predictions to original scale ----
train_pred_true_scale = scaler_y.inverse_transform(df_train_pred)
val_pred_true_scale = scaler_y.inverse_transform(df_val_pred)

# ---- 6) Create DataFrames for unscaled predictions ----
output_cols = ['heave', 'pitch', 'pendulum']
df_train_pred_true_scale = pd.DataFrame(train_pred_true_scale, columns=[f"{col}_pred" for col in output_cols])
df_val_pred_true_scale = pd.DataFrame(val_pred_true_scale, columns=[f"{col}_pred" for col in output_cols])

# ---- 7) Inverse transform ground truth Y data ----
y_train_true_scale = scaler_y.inverse_transform(y_train_seq)
y_val_true_scale = scaler_y.inverse_transform(Y_val_seq)

# ---- 8) Create DataFrames for ground truth ----
y_train_df_true_scale = pd.DataFrame(y_train_true_scale, columns=output_cols)
y_val_df_true_scale = pd.DataFrame(y_val_true_scale, columns=output_cols)

# ---- 9) Compute Metrics ----
from sklearn.metrics import r2_score, mean_squared_error

# R² Scores
r2_scores_train = {
    col: r2_score(y_train_df_true_scale[col], df_train_pred_true_scale[f"{col}_pred"])
    for col in output_cols
}
r2_scores_val = {
    col: r2_score(y_val_df_true_scale[col], df_val_pred_true_scale[f"{col}_pred"])
    for col in output_cols
}

# RMSE
rmse_train = {
    col: np.sqrt(mean_squared_error(y_train_df_true_scale[col], df_train_pred_true_scale[f"{col}_pred"]))
    for col in output_cols
}
rmse_val = {
    col: np.sqrt(mean_squared_error(y_val_df_true_scale[col], df_val_pred_true_scale[f"{col}_pred"]))
    for col in output_cols
}

# ---- 10) Print Results ----
print("Training R² Scores:", r2_scores_train)
print("Validation R² Scores:", r2_scores_val)
print("Training RMSE:", rmse_train)
print("Validation RMSE:", rmse_val)


# Testing 

In [ ]:
# load Best model
model = load_model("model_checkpoint_eta_only_nb5_nf5_new_archt_eta_vel_acc_more_stopping_creteria.h5")


In [ ]:
input_cols = ['eta' , 'eta_velocity','eta_acceleration']
output_cols = ['heave', 'pitch', 'pendulum']
nb= 5
nf=5
batch_size = 64
cases=df_test_full['test_name'].unique()
cases 

In [ ]:
# Initialize an empty list to collect the results for each test case
results = []
for case in cases:
    # Filter the DataFrame for the current test case
    df_test_case = df_test_full[df_test_full['test_name'] == case].copy()

    test_input = df_test_case[input_cols]
    test_output= df_test_case[output_cols]

    # scale the data
    input_df_scaled = scaler_X_func_all(test_input)
    output_df_scaled = scaler_y_func(test_output)

    X_test_seq,y_test_seq= create_exog_sequences(
        input_df = input_df_scaled,
        output_df=output_df_scaled,
        input_cols= input_cols,
        output_cols = output_cols,
        nb = nb,
        nf= nf
    )


    # Remove extra samples at the end to match batch size
    def adjust_for_batch(X, y):
        n = (X.shape[0] // batch_size) * batch_size
        return X[:n], y[:n]

    X_test_seq, y_test_seq = adjust_for_batch(X_test_seq, y_test_seq)

    # Get the raw predictions for the test set
    model.reset_states()

    y_test_preds = model.predict(
        X_test_seq,
        batch_size=batch_size,
        verbose=1,
        
    )

    heave_pred_test, pitch_pred_test, pend_pred_test = [arr.squeeze(-1) for arr in y_test_preds]

    # combine into a single array or DataFrame

    y_test_pred_arr = np.vstack([heave_pred_test, pitch_pred_test, pend_pred_test]).T
    df_test_pred = pd.DataFrame(
        y_test_pred_arr,
        columns=['heave_pred', 'pitch_pred', 'pendulum_pred']
    )

    # get true data 

    y_test_df=pd.DataFrame(y_test_seq, columns=['heave', 'pitch', 'pendulum'])

    # scale back tp oridinal scale

    y_test_predict_true_scale= scaler_y.inverse_transform(df_test_pred)
    y_test_true_scale= scaler_y.inverse_transform(y_test_df)

    # Convert NumPy arrays back to DataFrames with column names

    df_test_pred_true_scale = pd.DataFrame(
        y_test_predict_true_scale, 
        columns=[f"{col}_pred" for col in output_cols]  
    )

    y_test_df_true_scale = pd.DataFrame(y_test_true_scale, columns= output_cols)

    # R2 score 
    r2_scores_test = {col: r2_score(y_test_df_true_scale[col], df_test_pred_true_scale[f"{col}_pred"]) for col in output_cols}
    # MSE on training data
    mse_test = {col: mean_squared_error(y_test_df_true_scale[col], df_test_pred_true_scale[f"{col}_pred"]) for col in output_cols}
    print("test_case :", case)
    print("Test R² Scores:", r2_scores_test)
    print("Test MSE:", mse_test)
    
    # Collect the results in a dictionary (one dictionary per case)
    result = {
        'test_case': case,
        'heave_r2': r2_scores_test.get('heave', None),
        'pitch_r2': r2_scores_test.get('pitch', None),
        'pendulum_r2': r2_scores_test.get('pendulum', None),
        'heave_mse': mse_test.get('heave', None),
        'pitch_mse': mse_test.get('pitch', None),
        'pendulum_mse': mse_test.get('pendulum', None),
    }
    
    # Append the result dictionary to the results list
    results.append(result)
    
    # save predictions and true values to CSV files
    y_test_df_true_scale.to_csv(f'Results/LSTM/Testing/LSTM_true_{case}.csv', index=False)
    df_test_pred_true_scale.to_csv(f'Results/LSTM/Testing/LSTM_pred_{case}.csv', index=False)
    
# create a DataFrame from the results list
df_test_results_initial_not_zero = pd.DataFrame(results)

In [ ]:
# Save metrics to a CSV file
df_test_results_initial_not_zero.to_csv(f'Results/LSTM/Testing/LSTMdf_test_results_initial_not_zero.csv', index=False)

# Compuatational Time and Co2 emissions

In [ ]:
# Import the codecarbon library and print its version
import codecarbon
print(codecarbon.__version__)

from codecarbon import EmissionsTracker



## Training

In [ ]:
# preparing the Training and Validation data for the model

input_cols = ['eta','eta_velocity','eta_acceleration']
output_cols = ['heave', 'pitch', 'pendulum']
nb= 5
nf=5
batch_size = 64
# Create 4 rows of zeros for each DataFrame
zero_input = pd.DataFrame(np.zeros((5, len(input_cols))), columns=input_cols)
zero_output = pd.DataFrame(np.zeros((5, len(output_cols))), columns=output_cols)
input = df_case_train[input_cols]
output= df_case_train[output_cols]

input=pd.concat([zero_input, input], ignore_index=True)
output=pd.concat([zero_input, output], ignore_index=True)
# scale the data
input_df_scaled = scaler_X_func_all(input)
output_df_scaled = scaler_y_func(output)

X_train_seq,y_train_seq= create_exog_sequences(
    input_df = input_df_scaled,
    output_df=output_df_scaled,
    input_cols= input_cols,
    output_cols = output_cols,
    nb = nb,
    nf= nf
)

val_input = df_case_val[input_cols]
val_output= df_case_val[output_cols]
# scale the data
val_input_df_scaled = scaler_X_func_all(val_input)
val_output_df_scaled = scaler_y_func(val_output)

X_val_seq , Y_val_seq = create_exog_sequences(
    input_df = val_input_df_scaled,
    output_df= val_output_df_scaled,
    input_cols= input_cols,
    output_cols = output_cols,
    nb = nb,
    nf= nf
)

#Remove extra samples at the end to match batch size
def adjust_for_batch(X, y):
    n = (X.shape[0] // batch_size) * batch_size
    return X[:n], y[:n]

X_train_seq, y_train_seq = adjust_for_batch(X_train_seq, y_train_seq)
X_val_seq, Y_val_seq = adjust_for_batch(X_val_seq, Y_val_seq)
    

In [ ]:
# --- Model Parameters ---

heave_LSTM = 64
pitch_LSTM = 64
pend_LSTM = 64

shared_LSTM = 64
dropout = 0.2


# --- Create input shape info ---
time_steps = nb + 1 + nf  # full window
n_features = len(input_cols)  


In [ ]:
process = psutil.Process()
start_time = time.time()  # Start time for measurement
initial_memory = process.memory_info().rss / (1024 ** 2)  # Memory in MB
# build model 

seq_in = Input(batch_shape=(batch_size, time_steps, n_features), name="sequence")

# --- 1. Per-output LSTM layers ---
heave_branch = LSTM(heave_LSTM,stateful=True, return_sequences=True,  batch_size=batch_size, name='heave_lstm')(seq_in)
heave_branch = Dropout(dropout)(heave_branch)

pitch_branch = LSTM(pitch_LSTM,stateful=True, return_sequences=True,batch_size=batch_size,  name='pitch_lstm')(seq_in)
pitch_branch = Dropout(dropout)(pitch_branch)
    
pend_branch = LSTM(pend_LSTM, stateful=True, return_sequences=True, batch_size=batch_size, name='pendulum_lstm')(seq_in)
pend_branch = Dropout(dropout)(pend_branch)

# --- 2. Concatenate outputs ---
combined = Concatenate(name='combined_lstm_concat')([heave_branch, pitch_branch, pend_branch])

# --- 3. Shared LSTM layer to learn coupling ---
shared_lstm = LSTM(shared_LSTM, stateful=True,return_sequences=False,batch_size=batch_size,  name='shared_lstm')(combined)
shared_lstm = Dropout(dropout)(shared_lstm)

# --- 4. Final Dense outputs ---
heave_out = Dense(1, activation='tanh', name='heave')(shared_lstm)
pitch_out = Dense(1, activation='tanh', name='pitch')(shared_lstm)
pendulum_out = Dense(1, activation='tanh', name='pendulum')(shared_lstm)

# --- Define the model ---
model = Model(inputs=seq_in, outputs=[heave_out, pitch_out, pendulum_out])

# complie model 
model.compile(
    optimizer='adam',
    loss={'heave':'mse','pitch':'mse','pendulum':'mse'},
    metrics={'heave':'mae','pitch':'mae','pendulum':'mae'}
)

# Train model
history = model.fit(
    X_train_seq,
    {
        'heave': y_train_seq[:, 0],
        'pitch': y_train_seq[:, 1],
        'pendulum': y_train_seq[:, 2]
    },
    epochs=10,
    batch_size=batch_size,
    shuffle=False,
    validation_data=(
        X_val_seq,
        {
            'heave': Y_val_seq[:, 0],
            'pitch': Y_val_seq[:, 1],
            'pendulum': Y_val_seq[:, 2]
        }
    ),
    verbose=1,
    callbacks=[ResetStatesCallback()]
)

# Measure memory usage and time after prediction
end_time = time.time()  # End time for measurement
final_memory = process.memory_info().rss / (1024 ** 2)  # Memory in MB
# Print results
print(f"Time taken : {end_time - start_time} seconds")
print(f"Memory used: {final_memory - initial_memory} MB")

In [ ]:
tracker = EmissionsTracker(
    output_dir="Results/co2",       # Custom folder
    output_file="lstm_train.csv"    # Custom filename
)

tracker.start()
# build model 

seq_in = Input(batch_shape=(batch_size, time_steps, n_features), name="sequence")

# --- 1. Per-output LSTM layers ---
heave_branch = LSTM(heave_LSTM,stateful=True, return_sequences=True,  batch_size=batch_size, name='heave_lstm')(seq_in)
heave_branch = Dropout(dropout)(heave_branch)

pitch_branch = LSTM(pitch_LSTM,stateful=True, return_sequences=True,batch_size=batch_size,  name='pitch_lstm')(seq_in)
pitch_branch = Dropout(dropout)(pitch_branch)
    
pend_branch = LSTM(pend_LSTM, stateful=True, return_sequences=True, batch_size=batch_size, name='pendulum_lstm')(seq_in)
pend_branch = Dropout(dropout)(pend_branch)

# --- 2. Concatenate outputs ---
combined = Concatenate(name='combined_lstm_concat')([heave_branch, pitch_branch, pend_branch])

# --- 3. Shared LSTM layer to learn coupling ---
shared_lstm = LSTM(shared_LSTM, stateful=True,return_sequences=False,batch_size=batch_size,  name='shared_lstm')(combined)
shared_lstm = Dropout(dropout)(shared_lstm)

# --- 4. Final Dense outputs ---
heave_out = Dense(1, activation='tanh', name='heave')(shared_lstm)
pitch_out = Dense(1, activation='tanh', name='pitch')(shared_lstm)
pendulum_out = Dense(1, activation='tanh', name='pendulum')(shared_lstm)

# --- Define the model ---
model = Model(inputs=seq_in, outputs=[heave_out, pitch_out, pendulum_out])

# complie model 
model.compile(
    optimizer='adam',
    loss={'heave':'mse','pitch':'mse','pendulum':'mse'},
    metrics={'heave':'mae','pitch':'mae','pendulum':'mae'}
)

# Train model
history = model.fit(
    X_train_seq,
    {
        'heave': y_train_seq[:, 0],
        'pitch': y_train_seq[:, 1],
        'pendulum': y_train_seq[:, 2]
    },
    epochs=10,
    batch_size=batch_size,
    shuffle=False,
    validation_data=(
        X_val_seq,
        {
            'heave': Y_val_seq[:, 0],
            'pitch': Y_val_seq[:, 1],
            'pendulum': Y_val_seq[:, 2]
        }
    ),
    verbose=1,
    callbacks=[ResetStatesCallback()]
)

# Stop the tracker and retrieve the estimated emissions
emissions = tracker.stop()

## prediction

In [ ]:
# load Best model
model = load_model("model_checkpoint_eta_only_nb5_nf5_new_archt_eta_vel_acc_more_stopping_creteria.h5")


In [ ]:
case_test='Tp6p8s_Hs1m'
df_case_test=df_train_full[df_train_full['test_name']==case_test].reset_index(drop=True)
input_cols = ['eta','eta_velocity','eta_acceleration']
output_cols = ['heave', 'pitch', 'pendulum']
nb= 5
nf=5
batch_size = 64
# Create 4 rows of zeros for each DataFrame
zero_input = pd.DataFrame(np.zeros((5, len(input_cols))), columns=input_cols)
zero_output = pd.DataFrame(np.zeros((5, len(output_cols))), columns=output_cols)

input = df_case_test[input_cols]
output= df_case_test[output_cols]

input=pd.concat([zero_input, input], ignore_index=True)
output=pd.concat([zero_input, output], ignore_index=True)
# scale the data
input_df_scaled = scaler_X_func_all(input)
output_df_scaled = scaler_y_func(output)

X_test_seq,y_test_seq= create_exog_sequences(
    input_df = input_df_scaled,
    output_df=output_df_scaled,
    input_cols= input_cols,
    output_cols = output_cols,
    nb = nb,
    nf= nf
)
#Remove extra samples at the end to match batch size
def adjust_for_batch(X, y):
    n = (X.shape[0] // batch_size) * batch_size
    return X[:n], y[:n]

X_test_seq, y_test_seq = adjust_for_batch(X_test_seq, y_test_seq)



In [ ]:
process = psutil.Process()
start_time = time.time()  # Start time for measurement
initial_memory = process.memory_info().rss / (1024 ** 2)  # Memory in MB
# predictoins 
model.reset_states()

y_test_preds = model.predict(
        X_test_seq,
        batch_size=batch_size,
        verbose=1,
            )

# Measure memory usage and time after prediction
end_time = time.time()  # End time for measurement
final_memory = process.memory_info().rss / (1024 ** 2)  # Memory in MB
# Print results
print(f"Time taken : {end_time - start_time} seconds")
print(f"Memory used: {final_memory - initial_memory} MB")

In [ ]:
tracker = EmissionsTracker(
    output_dir="Results/co2",       # Custom folder
    output_file="lstm_pred.csv"    # Custom filename
)

tracker.start()
# predictoins 
model.reset_states()

y_test_preds = model.predict(
        X_test_seq,
        batch_size=batch_size,
        verbose=1,
            )

# Stop the tracker and retrieve the estimated emissions
emissions = tracker.stop()

In [ ]:

heave_pred_test, pitch_pred_test, pend_pred_test = [arr.squeeze(-1) for arr in y_test_preds]

# combine into a single array or DataFrame

y_test_pred_arr = np.vstack([heave_pred_test, pitch_pred_test, pend_pred_test]).T
df_test_pred = pd.DataFrame(
    y_test_pred_arr,
    columns=['heave_pred', 'pitch_pred', 'pendulum_pred']
)
 # get true data 

y_test_df=pd.DataFrame(y_test_seq, columns=['heave', 'pitch', 'pendulum'])

# scale back tp oridinal scale

y_test_predict_true_scale= scaler_y.inverse_transform(df_test_pred)
y_test_true_scale= scaler_y.inverse_transform(y_test_df)

# Convert NumPy arrays back to DataFrames with column names

df_test_pred_true_scale = pd.DataFrame(
    y_test_predict_true_scale, 
    columns=[f"{col}_pred" for col in output_cols]  
)

y_test_df_true_scale = pd.DataFrame(y_test_true_scale, columns= output_cols)

df_test_pred_true_scale.to_csv('Results/testing for comparison/lstm_pred.csv')
y_test_df_true_scale.to_csv('Results/testing for comparison/lstm_true.csv')


# Data senstivety Test


In [ ]:
input_cols = ['eta','eta_velocity','eta_acceleration']
output_cols = ['heave', 'pitch', 'pendulum']
nb= 5
nf=5
batch_size = 64
    

In [ ]:
# --- Model Parameters ---

heave_LSTM = 64
pitch_LSTM = 64
pend_LSTM = 64

shared_LSTM = 64
dropout = 0.2


# --- Create input shape info ---
time_steps = nb + 1 + nf  # full window
n_features = len(input_cols)  


In [ ]:
#preparing the Training Data
# Create 4 rows of zeros for each DataFrame
zero_input = pd.DataFrame(np.zeros((5, len(input_cols))), columns=input_cols)
zero_output = pd.DataFrame(np.zeros((5, len(output_cols))), columns=output_cols)

input = df_case_train[input_cols]
output= df_case_train[output_cols]

input=pd.concat([zero_input, input], ignore_index=True)
output=pd.concat([zero_input, output], ignore_index=True)
# scale the data
input_df_scaled = scaler_X_func_all(input)
output_df_scaled = scaler_y_func(output)

X_train_seq,y_train_seq= create_exog_sequences(
    input_df = input_df_scaled,
    output_df=output_df_scaled,
    input_cols= input_cols,
    output_cols = output_cols,
    nb = nb,
    nf= nf
)


In [ ]:
lenght=np.array([0.002,0.004,0.008,0.01 , 0.1 , 0.2 , 0.3 , 0.4 , 0.5 , 0.6 , 0.7 , 0.8 , 0.9,1])*X_train_seq.shape[0]
lenght=lenght.astype(int)
lenght

In [ ]:
# prepare validation data
val_input = df_case_val[input_cols]
val_output= df_case_val[output_cols]
# scale the data
val_input_df_scaled = scaler_X_func_all(val_input)
val_output_df_scaled = scaler_y_func(val_output)

X_val_seq , Y_val_seq = create_exog_sequences(
    input_df = val_input_df_scaled,
    output_df= val_output_df_scaled,
    input_cols= input_cols,
    output_cols = output_cols,
    nb = nb,
    nf= nf
)

#Remove extra samples at the end to match batch size
def adjust_for_batch(X, y):
    n = (X.shape[0] // batch_size) * batch_size
    return X[:n], y[:n]

X_val_seq_full, Y_val_seq_full = adjust_for_batch(X_val_seq, Y_val_seq)

In [ ]:
lenght_val=np.array([0.005,0.005,0.008,0.01 , 0.1 , 0.2 , 0.3 , 0.4 , 0.5 , 0.6 , 0.7 , 0.8 , 0.9,1])*X_val_seq.shape[0]
lenght_val=lenght_val.astype(int)
lenght_val

In [ ]:
# prepare test data
test_input = df_case_test[input_cols]
test_output= df_case_test[output_cols]
# scale the data
test_input_df_scaled = scaler_X_func_all(test_input)
test_output_df_scaled = scaler_y_func(test_output)

X_test_seq , Y_test_seq = create_exog_sequences(
    input_df = test_input_df_scaled,
    output_df= test_output_df_scaled,
    input_cols= input_cols,
    output_cols = output_cols,
    nb = nb,
    nf= nf
)

#Remove extra samples at the end to match batch size
def adjust_for_batch(X, y):
    n = (X.shape[0] // batch_size) * batch_size
    return X[:n], y[:n]

X_test_seq, Y_test_seq = adjust_for_batch(X_test_seq, Y_test_seq)

In [ ]:
# Initialize an empty list to collect the results for each test case
results = []
for l in [0,1,2,3]:
    
    # build model 

    seq_in = Input(batch_shape=(batch_size, time_steps, n_features), name="sequence")

    # --- 1. Per-output LSTM layers ---
    heave_branch = LSTM(heave_LSTM,stateful=True, return_sequences=True,  batch_size=batch_size, name='heave_lstm')(seq_in)
    heave_branch = Dropout(dropout)(heave_branch)

    pitch_branch = LSTM(pitch_LSTM,stateful=True, return_sequences=True,batch_size=batch_size,  name='pitch_lstm')(seq_in)
    pitch_branch = Dropout(dropout)(pitch_branch)
        
    pend_branch = LSTM(pend_LSTM, stateful=True, return_sequences=True, batch_size=batch_size, name='pendulum_lstm')(seq_in)
    pend_branch = Dropout(dropout)(pend_branch)

    # --- 2. Concatenate outputs ---
    combined = Concatenate(name='combined_lstm_concat')([heave_branch, pitch_branch, pend_branch])

    # --- 3. Shared LSTM layer to learn coupling ---
    shared_lstm = LSTM(shared_LSTM, stateful=True,return_sequences=False,batch_size=batch_size,  name='shared_lstm')(combined)
    shared_lstm = Dropout(dropout)(shared_lstm)

    # --- 4. Final Dense outputs ---
    heave_out = Dense(1, activation='tanh', name='heave')(shared_lstm)
    pitch_out = Dense(1, activation='tanh', name='pitch')(shared_lstm)
    pendulum_out = Dense(1, activation='tanh', name='pendulum')(shared_lstm)

    # --- Define the model ---
    model_data_test = Model(inputs=seq_in, outputs=[heave_out, pitch_out, pendulum_out])


    
    X_train_seq_adjusted=X_train_seq[0:lenght[l],:,:]
    y_train_seq_adjusted=y_train_seq[0:lenght[l],:]
    
    X_val_seq_adjusted=X_val_seq[0:lenght_val[l],:,:]
    Y_val_seq_adjusted=Y_val_seq[0:lenght_val[l],:]
    
    
    X_train_seq_adjusted, y_train_seq_adjusted = adjust_for_batch(X_train_seq_adjusted, y_train_seq_adjusted)
    X_val_seq_adjusted, Y_val_seq_adjusted = adjust_for_batch(X_val_seq_adjusted, Y_val_seq_adjusted)


    
    
    # compile the model
    model_data_test.compile (optimizer='adam',
    loss={'heave':'mse','pitch':'mse','pendulum':'mse'},
    metrics={'heave':'mae','pitch':'mae','pendulum':'mae'}
)

    
    # prepare training data 
    print(f'-----Preprocessing case of lenght {str(l)} ----')
   
    print("NaNs in X_train_seq_adjusted:", np.any(np.isnan(X_train_seq_adjusted)))
    print("NaNs in y_train_seq_adjusted:", np.any(np.isnan(y_train_seq_adjusted)))
    print("NaNs in X_val_seq_adjusted:", np.any(np.isnan(X_val_seq_adjusted)))
    print("NaNs in Y_val_seq_adjusted:", np.any(np.isnan(Y_val_seq_adjusted)))
   
    
    # ============================
    print(f'-----Training model----')
    # Train model
    history = model_data_test.fit(
    X_train_seq_adjusted,
    {
        'heave': y_train_seq_adjusted[:, 0],
        'pitch': y_train_seq_adjusted[:, 1],
        'pendulum': y_train_seq_adjusted[:, 2]
    },
    epochs=500,
    batch_size=batch_size,
    shuffle=False,
    validation_data=(
        X_val_seq_adjusted,
        {
            'heave': Y_val_seq_adjusted[:, 0],
            'pitch': Y_val_seq_adjusted[:, 1],
            'pendulum': Y_val_seq_adjusted[:, 2]
        }
    ),
    verbose=1,
    callbacks=[ResetStatesCallback(), early_stopping_more]
        )   
    
    
        
    # Get the raw predictions for the training set
    model_data_test.reset_states()
    print(f'-----predicting on training data----')
    
    y_train_preds = model_data_test.predict(
        X_train_seq_adjusted,
        batch_size=batch_size,
        verbose=1,
        
    )

    # 3) y_train_preds is a list of three arrays (one per output head)

    heave_pred, pitch_pred, pend_pred = [arr.squeeze(-1) for arr in y_train_preds]

    # 4) (Optional) combine into a single array or DataFrame


    y_train_pred_arr = np.vstack([heave_pred, pitch_pred, pend_pred]).T
    df_train_pred = pd.DataFrame(
        y_train_pred_arr,
        columns=['heave_pred', 'pitch_pred', 'pendulum_pred']
    )

    print(df_train_pred.head())

    # 1) Reset & warm‑start again, just like for evaluation
    model_data_test.reset_states()

    print(f'-----predicting on testing data----')

    # 2) Get the raw predictions for the testing set
    y_test_preds = model_data_test.predict(
    X_test_seq,
    batch_size=batch_size,
    verbose=1,

    )

    # 3) y_val_preds is a list of three arrays (one per output head)
    heave_pred, pitch_pred, pend_pred = [arr.squeeze(-1) for arr in y_test_preds]

    # 4) (Optional) combine into a single array or DataFrame
    y_test_pred_arr = np.vstack([heave_pred, pitch_pred, pend_pred]).T
    df_y_test_pred = pd.DataFrame(
    y_test_pred_arr,
    columns=['heave_pred', 'pitch_pred', 'pendulum_pred']
    )

    print(df_y_test_pred.head())
    
    
    
    # 1) Reset & warm‑start again, just like for evaluation
    model_data_test.reset_states()

    print(f'-----predicting on validation data----')

    # 2) Get the raw predictions for the validation set
    y_val_preds = model_data_test.predict(
    X_val_seq_adjusted,
    batch_size=batch_size,
    verbose=1,

    )

    # 3) y_val_preds is a list of three arrays (one per output head)
    heave_pred, pitch_pred, pend_pred = [arr.squeeze(-1) for arr in y_val_preds]

    # 4) (Optional) combine into a single array or DataFrame
    y_val_pred_arr = np.vstack([heave_pred, pitch_pred, pend_pred]).T
    df_val_pred = pd.DataFrame(
    y_val_pred_arr,
    columns=['heave_pred', 'pitch_pred', 'pendulum_pred']
    )

    print(df_val_pred.head())
    
               
    
    # scale back the predictions to original scale
    train_pred_true_scale= scaler_y.inverse_transform(df_train_pred)
    test_pred_true_scale= scaler_y.inverse_transform(df_y_test_pred)
    val_pred_true_scale= scaler_y.inverse_transform(df_val_pred)
    output_cols = ['heave', 'pitch', 'pendulum']  # replace with your actual target column names

    # Convert NumPy arrays back to DataFrames with column names
    df_train_pred_true_scale = pd.DataFrame(
    train_pred_true_scale, 
    columns=[f"{col}_pred" for col in output_cols]  # creates columns: 'heave_pred', 'pitch_pred', etc.
    )

    df_test_pred_true_scale = pd.DataFrame(
    test_pred_true_scale,
    columns=[f"{col}_pred" for col in output_cols])
    
    df_val_pred_true_scale = pd.DataFrame(
    val_pred_true_scale,
    columns=[f"{col}_pred" for col in output_cols])

    # get df with the traing and validation data at corrensponding time steps
    y_train_df=pd.DataFrame(y_train_seq_adjusted, columns=['heave', 'pitch', 'pendulum'])
    y_val_df=pd.DataFrame(Y_val_seq_adjusted, columns=['heave', 'pitch', 'pendulum'])
    y_test_df=pd.DataFrame(Y_test_seq, columns=['heave', 'pitch', 'pendulum'])

    # scale back tp oridinal scale
    y_train_true_scale= scaler_y.inverse_transform(y_train_df)
    y_test_true_scale= scaler_y.inverse_transform(y_test_df)
    y_val_true_scale= scaler_y.inverse_transform(y_val_df)

    # convert to df again
    y_train_df_true_scale = pd.DataFrame(y_train_true_scale, columns=['heave', 'pitch', 'pendulum'])
    y_test_df_true_scale = pd.DataFrame(y_test_true_scale, columns=['heave', 'pitch', 'pendulum'])
    y_val_df_true_scale = pd.DataFrame(y_val_true_scale, columns=['heave', 'pitch', 'pendulum'])
    

    # Define your output column names
    output_cols = ['heave', 'pitch', 'pendulum']
    # R2 score on training data
    r2_scores_train = {col: r2_score(y_train_df_true_scale[col], df_train_pred_true_scale[f"{col}_pred"]) for col in output_cols}
    print("Training R² Scores:", r2_scores_train)
    # R2 score on validation data
    r2_scores_val = {col: r2_score(y_val_df_true_scale[col], df_val_pred_true_scale[f"{col}_pred"]) for col in output_cols}
    # R2 score on full validation data
    r2_scores_test = {col: r2_score(y_test_df_true_scale[col], df_test_pred_true_scale[f"{col}_pred"]) for col in output_cols}
    print("Testing R² Scores:", r2_scores_test)
    # MSE on training data
    mse_train = {col: mean_squared_error(y_train_df_true_scale[col], df_train_pred_true_scale[f"{col}_pred"]) for col in output_cols}
    print("Training RMSE:", mse_train)
    # MSE on validation data
    mse_val = {col: mean_squared_error(y_val_df_true_scale[col], df_val_pred_true_scale[f"{col}_pred"]) for col in output_cols}
    # MSE on full validation data
    mse_test = {col: mean_squared_error(y_test_df_true_scale[col], df_test_pred_true_scale[f"{col}_pred"]) for col in output_cols}
    print("Testing RMSE:", mse_test)
    
    # Collect the results in a dictionaru
    result = {
        'data_lenght': lenght[l],
        'train_heave_r2': r2_scores_train.get('heave', None),
        'train_pitch_r2': r2_scores_train.get('pitch', None),
        'train_pendulum_r2': r2_scores_train.get('pendulum', None),
        'train_heave_mse': mse_train.get('heave', None),
        'train_pitch_mse': mse_train.get('pitch', None),
        'train_pendulum_mse': mse_train.get('pendulum', None),
        'val_heave_r2': r2_scores_val.get('heave', None),
        'val_pitch_r2': r2_scores_val.get('pitch', None),
        'val_pendulum_r2': r2_scores_val.get('pendulum', None),
        'val_heave_mse': mse_val.get('heave', None),
        'val_pitch_mse': mse_val.get('pitch', None),
        'val_pendulum_mse': mse_val.get('pendulum', None),
        
        'test_heave_r2': r2_scores_test.get('heave', None),
        'test_pitch_r2': r2_scores_test.get('pitch', None),
        'test_pendulum_r2': r2_scores_test.get('pendulum', None),
        'test_heave_mse': mse_test.get('heave', None),
        'test_pitch_mse': mse_test.get('pitch', None),
        'test_pendulum_mse': mse_test.get('pendulum', None),
                
    }
    
    # Append the result dictionary to the results list
    results.append(result)
    


In [ ]:
# create a DataFrame from the results list
df_data_test_results = pd.DataFrame(results) 
df_data_test_results.to_csv('Results/LSTM/data test/df_data_test_results_lstm_final_ver2.csv', index=False)

In [ ]:
df_data_test_results

# dt senstivety Test

In [ ]:
input_cols = ['eta','eta_velocity','eta_acceleration']
output_cols = ['heave', 'pitch', 'pendulum']
nb= 5
nf=5
batch_size = 64
    

In [ ]:
# --- Model Parameters ---

heave_LSTM = 64
pitch_LSTM = 64
pend_LSTM = 64

shared_LSTM = 64
dropout = 0.2


# --- Create input shape info ---
time_steps = nb + 1 + nf  # full window
n_features = len(input_cols)  


In [ ]:
# dt steps
steps=[1,2,3,4,5,6]


In [ ]:
# prepare Training Data
# Create 4 rows of zeros for each DataFrame
zero_input = pd.DataFrame(np.zeros((5, len(input_cols))), columns=input_cols)
zero_output = pd.DataFrame(np.zeros((5, len(output_cols))), columns=output_cols)

input = df_case_train[input_cols]
output= df_case_train[output_cols]

input=pd.concat([zero_input, input], ignore_index=True)
output=pd.concat([zero_input, output], ignore_index=True)
# scale the data
input_df_scaled = scaler_X_func_all(input)
output_df_scaled = scaler_y_func(output)

X_train_seq,y_train_seq= create_exog_sequences(
    input_df = input_df_scaled,
    output_df=output_df_scaled,
    input_cols= input_cols,
    output_cols = output_cols,
    nb = nb,
    nf= nf
)


In [ ]:
# prepare validation data
val_input = df_case_val[input_cols]
val_output= df_case_val[output_cols]
# scale the data
val_input_df_scaled = scaler_X_func_all(val_input)
val_output_df_scaled = scaler_y_func(val_output)

X_val_seq , Y_val_seq = create_exog_sequences(
    input_df = val_input_df_scaled,
    output_df= val_output_df_scaled,
    input_cols= input_cols,
    output_cols = output_cols,
    nb = nb,
    nf= nf
)

#Remove extra samples at the end to match batch size
def adjust_for_batch(X, y):
    n = (X.shape[0] // batch_size) * batch_size
    return X[:n], y[:n]

X_val_seq_full, Y_val_seq_full = adjust_for_batch(X_val_seq, Y_val_seq)

In [ ]:
# prepare test data
test_input = df_case_test[input_cols]
test_output= df_case_test[output_cols]
# scale the data
test_input_df_scaled = scaler_X_func_all(test_input)
test_output_df_scaled = scaler_y_func(test_output)

X_test_seq , Y_test_seq = create_exog_sequences(
    input_df = test_input_df_scaled,
    output_df= test_output_df_scaled,
    input_cols= input_cols,
    output_cols = output_cols,
    nb = nb,
    nf= nf
)



In [ ]:
# Initialize an empty list to collect the results for each test case
results = []
for step in steps:
    
    dt=0.05*step    
    # build model 

    seq_in = Input(batch_shape=(batch_size, time_steps, n_features), name="sequence")

    # --- 1. Per-output LSTM layers ---
    heave_branch = LSTM(heave_LSTM,stateful=True, return_sequences=True,  batch_size=batch_size, name='heave_lstm')(seq_in)
    heave_branch = Dropout(dropout)(heave_branch)

    pitch_branch = LSTM(pitch_LSTM,stateful=True, return_sequences=True,batch_size=batch_size,  name='pitch_lstm')(seq_in)
    pitch_branch = Dropout(dropout)(pitch_branch)
        
    pend_branch = LSTM(pend_LSTM, stateful=True, return_sequences=True, batch_size=batch_size, name='pendulum_lstm')(seq_in)
    pend_branch = Dropout(dropout)(pend_branch)

    # --- 2. Concatenate outputs ---
    combined = Concatenate(name='combined_lstm_concat')([heave_branch, pitch_branch, pend_branch])

    # --- 3. Shared LSTM layer to learn coupling ---
    shared_lstm = LSTM(shared_LSTM, stateful=True,return_sequences=False,batch_size=batch_size,  name='shared_lstm')(combined)
    shared_lstm = Dropout(dropout)(shared_lstm)

    # --- 4. Final Dense outputs ---
    heave_out = Dense(1, activation='tanh', name='heave')(shared_lstm)
    pitch_out = Dense(1, activation='tanh', name='pitch')(shared_lstm)
    pendulum_out = Dense(1, activation='tanh', name='pendulum')(shared_lstm)

    # --- Define the model ---
    model_data_test = Model(inputs=seq_in, outputs=[heave_out, pitch_out, pendulum_out])


    
    X_train_seq_adjusted=X_train_seq[::step,:, :]
    y_train_seq_adjusted=y_train_seq[::step,:]
    
    X_test_seq_adjusted=X_test_seq[::step,:, :]
    Y_test_seq_adjusted=Y_test_seq[::step,:]
    
    X_val_seq_adjusted=X_val_seq[::step,:, :]
    Y_val_seq_adjusted=Y_val_seq[::step,:]
    
    
    X_train_seq_adjusted, y_train_seq_adjusted = adjust_for_batch(X_train_seq_adjusted, y_train_seq_adjusted)
    X_val_seq_adjusted, Y_val_seq_adjusted = adjust_for_batch(X_val_seq_adjusted, Y_val_seq_adjusted)
    X_test_seq_adjusted, Y_test_seq_adjusted = adjust_for_batch(X_test_seq_adjusted, Y_test_seq_adjusted)


    
    
    # compile the model
    model_data_test.compile (optimizer='adam',
    loss={'heave':'mse','pitch':'mse','pendulum':'mse'},
    metrics={'heave':'mae','pitch':'mae','pendulum':'mae'}
)

    
    # prepare training data 
    print(f'-----Preprocessing case of dt {str(dt)} ----')
   
    print("NaNs in X_train_seq_adjusted:", np.any(np.isnan(X_train_seq_adjusted)))
    print("NaNs in y_train_seq_adjusted:", np.any(np.isnan(y_train_seq_adjusted)))
    print("NaNs in X_val_seq_adjusted:", np.any(np.isnan(X_val_seq_adjusted)))
    print("NaNs in Y_val_seq_adjusted:", np.any(np.isnan(Y_val_seq_adjusted)))
   
    
    # ============================
    print(f'-----Training model----')
    # Train model
    history = model_data_test.fit(
    X_train_seq_adjusted,
    {
        'heave': y_train_seq_adjusted[:, 0],
        'pitch': y_train_seq_adjusted[:, 1],
        'pendulum': y_train_seq_adjusted[:, 2]
    },
    epochs=500,
    batch_size=batch_size,
    shuffle=False,
    validation_data=(
        X_val_seq_adjusted,
        {
            'heave': Y_val_seq_adjusted[:, 0],
            'pitch': Y_val_seq_adjusted[:, 1],
            'pendulum': Y_val_seq_adjusted[:, 2]
        }
    ),
    verbose=1,
    callbacks=[ResetStatesCallback(), early_stopping_more]
        )   
    
    
        
    # Get the raw predictions for the training set
    model_data_test.reset_states()
    print(f'-----predicting on training data----')
    
    y_train_preds = model_data_test.predict(
        X_train_seq_adjusted,
        batch_size=batch_size,
        verbose=1,
        
    )

    # 3) y_train_preds is a list of three arrays (one per output head)

    heave_pred, pitch_pred, pend_pred = [arr.squeeze(-1) for arr in y_train_preds]

    # 4) (Optional) combine into a single array or DataFrame


    y_train_pred_arr = np.vstack([heave_pred, pitch_pred, pend_pred]).T
    df_train_pred = pd.DataFrame(
        y_train_pred_arr,
        columns=['heave_pred', 'pitch_pred', 'pendulum_pred']
    )

    print(df_train_pred.head())

    # 1) Reset & warm‑start again, just like for evaluation
    model_data_test.reset_states()

    print(f'-----predicting on testing data----')

    # 2) Get the raw predictions for the testing set
    y_test_preds = model_data_test.predict(
    X_test_seq_adjusted,
    batch_size=batch_size,
    verbose=1,

    )

    # 3) y_val_preds is a list of three arrays (one per output head)
    heave_pred, pitch_pred, pend_pred = [arr.squeeze(-1) for arr in y_test_preds]

    # 4) (Optional) combine into a single array or DataFrame
    y_test_pred_arr = np.vstack([heave_pred, pitch_pred, pend_pred]).T
    df_y_test_pred = pd.DataFrame(
    y_test_pred_arr,
    columns=['heave_pred', 'pitch_pred', 'pendulum_pred']
    )

    print(df_y_test_pred.head())
    
    
    
    # 1) Reset & warm‑start again, just like for evaluation
    model_data_test.reset_states()

    print(f'-----predicting on validation data----')

    # 2) Get the raw predictions for the validation set
    y_val_preds = model_data_test.predict(
    X_val_seq_adjusted,
    batch_size=batch_size,
    verbose=1,

    )

    # 3) y_val_preds is a list of three arrays (one per output head)
    heave_pred, pitch_pred, pend_pred = [arr.squeeze(-1) for arr in y_val_preds]

    # 4) (Optional) combine into a single array or DataFrame
    y_val_pred_arr = np.vstack([heave_pred, pitch_pred, pend_pred]).T
    df_val_pred = pd.DataFrame(
    y_val_pred_arr,
    columns=['heave_pred', 'pitch_pred', 'pendulum_pred']
    )

    print(df_val_pred.head())
    
               
    
    # scale back the predictions to original scale
    train_pred_true_scale= scaler_y.inverse_transform(df_train_pred)
    test_pred_true_scale= scaler_y.inverse_transform(df_y_test_pred)
    val_pred_true_scale= scaler_y.inverse_transform(df_val_pred)
    output_cols = ['heave', 'pitch', 'pendulum']  # replace with your actual target column names

    # Convert NumPy arrays back to DataFrames with column names
    df_train_pred_true_scale = pd.DataFrame(
    train_pred_true_scale, 
    columns=[f"{col}_pred" for col in output_cols]  # creates columns: 'heave_pred', 'pitch_pred', etc.
    )

    df_test_pred_true_scale = pd.DataFrame(
    test_pred_true_scale,
    columns=[f"{col}_pred" for col in output_cols])
    
    df_val_pred_true_scale = pd.DataFrame(
    val_pred_true_scale,
    columns=[f"{col}_pred" for col in output_cols])

    # get df with the traing and validation data at corrensponding time steps
    y_train_df=pd.DataFrame(y_train_seq_adjusted, columns=['heave', 'pitch', 'pendulum'])
    y_val_df=pd.DataFrame(Y_val_seq_adjusted, columns=['heave', 'pitch', 'pendulum'])
    y_test_df=pd.DataFrame(Y_test_seq_adjusted, columns=['heave', 'pitch', 'pendulum'])

    # scale back tp oridinal scale
    y_train_true_scale= scaler_y.inverse_transform(y_train_df)
    y_test_true_scale= scaler_y.inverse_transform(y_test_df)
    y_val_true_scale= scaler_y.inverse_transform(y_val_df)

    # convert to df again
    y_train_df_true_scale = pd.DataFrame(y_train_true_scale, columns=['heave', 'pitch', 'pendulum'])
    y_test_df_true_scale = pd.DataFrame(y_test_true_scale, columns=['heave', 'pitch', 'pendulum'])
    y_val_df_true_scale = pd.DataFrame(y_val_true_scale, columns=['heave', 'pitch', 'pendulum'])
    

    # Define your output column names
    output_cols = ['heave', 'pitch', 'pendulum']
    # R2 score on training data
    r2_scores_train = {col: r2_score(y_train_df_true_scale[col], df_train_pred_true_scale[f"{col}_pred"]) for col in output_cols}
    print("Training R² Scores:", r2_scores_train)
    # R2 score on validation data
    r2_scores_val = {col: r2_score(y_val_df_true_scale[col], df_val_pred_true_scale[f"{col}_pred"]) for col in output_cols}
    # R2 score on full validation data
    r2_scores_test = {col: r2_score(y_test_df_true_scale[col], df_test_pred_true_scale[f"{col}_pred"]) for col in output_cols}
    print("Testing R² Scores:", r2_scores_test)
    # MSE on training data
    mse_train = {col: mean_squared_error(y_train_df_true_scale[col], df_train_pred_true_scale[f"{col}_pred"]) for col in output_cols}
    print("Training RMSE:", mse_train)
    # MSE on validation data
    mse_val = {col: mean_squared_error(y_val_df_true_scale[col], df_val_pred_true_scale[f"{col}_pred"]) for col in output_cols}
    # MSE on full validation data
    mse_test = {col: mean_squared_error(y_test_df_true_scale[col], df_test_pred_true_scale[f"{col}_pred"]) for col in output_cols}
    print("Testing RMSE:", mse_test)
    
    # Collect the results in a dictionaru
    result = {
        'dt': dt,
        'train_heave_r2': r2_scores_train.get('heave', None),
        'train_pitch_r2': r2_scores_train.get('pitch', None),
        'train_pendulum_r2': r2_scores_train.get('pendulum', None),
        'train_heave_mse': mse_train.get('heave', None),
        'train_pitch_mse': mse_train.get('pitch', None),
        'train_pendulum_mse': mse_train.get('pendulum', None),
        'val_heave_r2': r2_scores_val.get('heave', None),
        'val_pitch_r2': r2_scores_val.get('pitch', None),
        'val_pendulum_r2': r2_scores_val.get('pendulum', None),
        'val_heave_mse': mse_val.get('heave', None),
        'val_pitch_mse': mse_val.get('pitch', None),
        'val_pendulum_mse': mse_val.get('pendulum', None),
        
        'test_heave_r2': r2_scores_test.get('heave', None),
        'test_pitch_r2': r2_scores_test.get('pitch', None),
        'test_pendulum_r2': r2_scores_test.get('pendulum', None),
        'test_heave_mse': mse_test.get('heave', None),
        'test_pitch_mse': mse_test.get('pitch', None),
        'test_pendulum_mse': mse_test.get('pendulum', None),
                
    }
    
    # Append the result dictionary to the results list
    results.append(result)
    


In [ ]:
# create a DataFrame from the results list
df_dt_test_results = pd.DataFrame(results) 
df_dt_test_results 


In [ ]:
df_dt_test_results.to_csv('Results/dt_tests/df_dt_test_results_lstm.csv', index=False)

# Noise Test

In [ ]:
# load Best model
model = load_model("model_checkpoint_eta_only_nb5_nf5_new_archt_eta_vel_acc_more_stopping_creteria.h5")


In [ ]:
# Load the noisy test data
df_case_test_noisy=pd.read_csv('Results/df_case_test_noisy.csv')
df_case_test_noisy

In [ ]:
input_cols = ['eta' , 'eta_velocity','eta_acceleration']
output_cols = ['heave', 'pitch', 'pendulum']
nb= 5
nf=5
batch_size = 64

In [ ]:
# preparing the Testing Data
input = df_case_test_noisy[input_cols]
output= df_case_test_noisy[output_cols]

# scale the data
input_df_scaled = scaler_X_func_all(input)
output_df_scaled = scaler_y_func(output)

X_test_seq,y_test_seq= create_exog_sequences(
    input_df = input_df_scaled,
    output_df=output_df_scaled,
    input_cols= input_cols,
    output_cols = output_cols,
    nb = nb,
    nf= nf
)
#Remove extra samples at the end to match batch size
def adjust_for_batch(X, y):
    n = (X.shape[0] // batch_size) * batch_size
    return X[:n], y[:n]

X_test_seq, y_test_seq = adjust_for_batch(X_test_seq, y_test_seq)


In [ ]:
# predictoins 
model.reset_states()

y_test_preds = model.predict(
        X_test_seq,
        batch_size=batch_size,
        verbose=1,
            )


In [ ]:

heave_pred, pitch_pred, pend_pred = [arr.squeeze(-1) for arr in y_test_preds]
y_test_pred_arr = np.vstack([heave_pred, pitch_pred, pend_pred]).T
# Create prediction DataFrames (scaled) ----
pred_cols = ['heave_pred', 'pitch_pred', 'pendulum_pred']

df_noise_test_pred = pd.DataFrame(y_test_pred_arr, columns=pred_cols)

# ---- 5) Inverse transform predictions to original scale ----
noise_test_pred_true_scale = scaler_y.inverse_transform(df_noise_test_pred)

# ---- 6) Create DataFrames for unscaled predictions ----

df_noise_test_pred_true_scale = pd.DataFrame(noise_test_pred_true_scale, columns=[f"{col}_pred" for col in output_cols])


# ---- 7) Inverse transform ground truth Y data ----
y_test_true_scale = scaler_y.inverse_transform(y_test_seq)

# ---- 8) Create DataFrames for ground truth ----
df_y_test_true_scale = pd.DataFrame(y_test_true_scale, columns=output_cols)

# ---- 9) Compute Metrics ----
from sklearn.metrics import r2_score, mean_squared_error

# R² Scores
r2_scores = {
    col: r2_score(df_y_test_true_scale[col], df_noise_test_pred_true_scale[f"{col}_pred"])
    for col in output_cols
}

# RMSE
rmse = {
    col: np.sqrt(mean_squared_error(df_y_test_true_scale[col], df_noise_test_pred_true_scale[f"{col}_pred"]))
    for col in output_cols
}


# ---- 10) Print Results ----
print("testing R² Scores:", r2_scores)
print("Testing RMSE:", rmse)


In [ ]:
# save the results
df_y_test_true_scale.to_csv('Results/noisy test/LSTM_true.csv')
df_noise_test_pred_true_scale.to_csv('Results/noisy test/LSTM_pred.csv')